In [1]:
# Version 5
# KNNImputer for all data

# Based
- https://www.kaggle.com/code/honganzhu/cmi-piu-competition?scriptVersionId=201912528 Version44 LB0.492

 If you find this notebook useful, please upvote this and the based one.

In [2]:
import numpy as np
import pandas as pd
import os
import re
from sklearn.base import clone
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import StratifiedKFold
from scipy.optimize import minimize
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import polars as pl
import polars.selectors as cs
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator, FormatStrFormatter, PercentFormatter
import seaborn as sns

from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim

from colorama import Fore, Style
from IPython.display import clear_output
import warnings
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import VotingRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.pipeline import Pipeline
from pytorch_tabnet.tab_model import TabNetRegressor
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', None)

pl.Config.set_tbl_rows(-1)
pl.Config.set_tbl_cols(-1)
pl.Config.set_fmt_str_lengths(10000)

polars.config.Config

In [3]:
SEED = 42
n_splits = 5

# Feature Engineering

- **Feature Selection**: The dataset contains features related to physical characteristics (e.g., BMI, Height, Weight), behavioral aspects (e.g., internet usage), and fitness data (e.g., endurance time). 
- **Categorical Feature Encoding**: Categorical features are mapped to numerical values using custom mappings for each unique category within the dataset. This ensures compatibility with machine learning algorithms that require numerical input.
- **Time Series Aggregation**: Time series statistics (e.g., mean, standard deviation) from the actigraphy data are computed and merged into the main dataset to create additional features for model training.


In [4]:
def process_file(id_folder, dirname):
    files = os.listdir(os.path.join(path, dirname, id_folder))
    if len(files) > 1:
        df_list = []
        for file in files:
            df = pd.read_parquet(os.path.join(dirname, id_folder, file))
            df_list.append(df)
        df = pd.concat(df_list, ignore_index=True)
    else:
        df = pd.read_parquet(os.path.join(dirname, id_folder, files[0]))
    
    df.drop('step', axis=1, inplace=True)

    describe_df = df.describe(percentiles=[0.01] + [i*0.5/10 + 0.05 for i in range(19)])
    count_mean = describe_df.loc['count', :].mean()
    describe_df.drop('count', axis=0, inplace=True)
    
    return np.append(describe_df.values.reshape(-1), count_mean), id_folder.split('=')[1]

def load_time_series(dirname) -> pd.DataFrame:
    ids = os.listdir(dirname)
    
    with ThreadPoolExecutor() as executor:
        results = list(tqdm(executor.map(lambda fname: process_file(fname, dirname), ids), total=len(ids)))
    
    stats, indexes = zip(*results)
    
    df = pd.DataFrame(stats, columns=[f"stat_{i}" for i in range(len(stats[0]))])
    df['id'] = indexes
    return df


class AutoEncoder(nn.Module):
    def __init__(self, input_dim, encoding_dim):
        super(AutoEncoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, encoding_dim*4),
            nn.ReLU(),
            nn.Linear(encoding_dim*4, encoding_dim*2),
            nn.ReLU(),
            nn.Linear(encoding_dim*2, encoding_dim),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim, encoding_dim*2),
            nn.ReLU(),
            nn.Linear(encoding_dim*2, encoding_dim*4),
            nn.ReLU(),
            nn.Linear(encoding_dim*4, input_dim),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded


def perform_autoencoder(df, encoding_dim=50, epochs=50, batch_size=32):
    scaler = StandardScaler()
    df_scaled = scaler.fit_transform(df)
    
    data_tensor = torch.FloatTensor(df_scaled)
    
    input_dim = data_tensor.shape[1]
    autoencoder = AutoEncoder(input_dim, encoding_dim)
    
    criterion = nn.MSELoss()
    optimizer = optim.Adam(autoencoder.parameters())
    
    for epoch in range(epochs):
        for i in range(0, len(data_tensor), batch_size):
            batch = data_tensor[i : i + batch_size]
            optimizer.zero_grad()
            reconstructed = autoencoder(batch)
            loss = criterion(reconstructed, batch)
            loss.backward()
            optimizer.step()
            
        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}]')
                 
    with torch.no_grad():
        encoded_data = autoencoder.encoder(data_tensor).numpy()
        
    df_encoded = pd.DataFrame(encoded_data, columns=[f'Enc_{i + 1}' for i in range(encoded_data.shape[1])])
    
    return df_encoded

def feature_engineering(df):
    df['BMI_Age'] = df['Physical-BMI'] * df['Basic_Demos-Age']
    df['Internet_Hours_Age'] = df['PreInt_EduHx-computerinternet_hoursday'] * df['Basic_Demos-Age']
    df['BMI_Internet_Hours'] = df['Physical-BMI'] * df['PreInt_EduHx-computerinternet_hoursday']
    df['BFP_BMI'] = df['BIA-BIA_Fat'] / df['BIA-BIA_BMI']
    df['FFMI_BFP'] = df['BIA-BIA_FFMI'] / df['BIA-BIA_Fat']
    df['FMI_BFP'] = df['BIA-BIA_FMI'] / df['BIA-BIA_Fat']
    df['LST_TBW'] = df['BIA-BIA_LST'] / df['BIA-BIA_TBW']
    df['BFP_BMR'] = df['BIA-BIA_Fat'] * df['BIA-BIA_BMR']
    df['BFP_DEE'] = df['BIA-BIA_Fat'] * df['BIA-BIA_DEE']
    df['BMR_Weight'] = df['BIA-BIA_BMR'] / df['Physical-Weight']
    df['DEE_Weight'] = df['BIA-BIA_DEE'] / df['Physical-Weight']
    df['SMM_Height'] = df['BIA-BIA_SMM'] / df['Physical-Height']
    df['Muscle_to_Fat'] = df['BIA-BIA_SMM'] / df['BIA-BIA_FMI']
    df['Hydration_Status'] = df['BIA-BIA_TBW'] / df['Physical-Weight']
    df['ICW_TBW'] = df['BIA-BIA_ICW'] / df['BIA-BIA_TBW']
    
    return df

In [5]:
path = "I:/Kaggle/child-mind-institute-problematic-internet-use/"

In [6]:
train = pd.read_csv(path + 'train.csv')
test = pd.read_csv(path + 'test.csv')
sample = pd.read_csv(path + 'sample_submission.csv')

In [7]:
train_ts = load_time_series(path + "series_train.parquet")
test_ts = load_time_series(path + "series_test.parquet")

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 16.36it/s]


In [8]:
concat_ts = pd.concat([train_ts, test_ts], ignore_index=True)
df_concat = concat_ts.drop('id', axis=1)

In [9]:
concat_ts_encoded = perform_autoencoder(df_concat, encoding_dim=64, epochs=1000, batch_size=128)

Epoch [10/1000], Loss: 0.7592]
Epoch [20/1000], Loss: 0.6980]
Epoch [30/1000], Loss: 0.5991]
Epoch [40/1000], Loss: 0.5749]
Epoch [50/1000], Loss: 0.5654]
Epoch [60/1000], Loss: 0.5673]
Epoch [70/1000], Loss: 0.5544]
Epoch [80/1000], Loss: 0.5543]
Epoch [90/1000], Loss: 0.5463]
Epoch [100/1000], Loss: 0.5453]
Epoch [110/1000], Loss: 0.5404]
Epoch [120/1000], Loss: 0.5379]
Epoch [130/1000], Loss: 0.5392]
Epoch [140/1000], Loss: 0.5348]
Epoch [150/1000], Loss: 0.5335]
Epoch [160/1000], Loss: 0.5335]
Epoch [170/1000], Loss: 0.5318]
Epoch [180/1000], Loss: 0.5335]
Epoch [190/1000], Loss: 0.5295]
Epoch [200/1000], Loss: 0.5319]
Epoch [210/1000], Loss: 0.5294]
Epoch [220/1000], Loss: 0.5289]
Epoch [230/1000], Loss: 0.5273]
Epoch [240/1000], Loss: 0.5271]
Epoch [250/1000], Loss: 0.5254]
Epoch [260/1000], Loss: 0.5247]
Epoch [270/1000], Loss: 0.5248]
Epoch [280/1000], Loss: 0.5259]
Epoch [290/1000], Loss: 0.5236]
Epoch [300/1000], Loss: 0.5255]
Epoch [310/1000], Loss: 0.5230]
Epoch [320/1000],

In [10]:
for col in concat_ts_encoded.columns:
    if concat_ts_encoded[col].std() == 0:
        concat_ts_encoded.drop(col, axis=1, inplace=True)

In [11]:
time_series_cols = concat_ts_encoded.columns.tolist()
concat_ts_encoded["id"] = concat_ts["id"]
train_ts_encoded = concat_ts_encoded.head(train_ts.shape[0]).reset_index(drop=True)
test_ts_encoded = concat_ts_encoded.tail(test_ts.shape[0]).reset_index(drop=True)
train_merge = pd.merge(train, train_ts_encoded, how="left", on='id')
test_merge = pd.merge(test, test_ts_encoded, how="left", on='id')

In [12]:
data_dict = pd.read_csv(path + 'data_dictionary.csv')
print(data_dict.shape)
data_dict

(81, 6)


,Instrument,Field,Description,Type,Values,Value Labels
0,Identifier,id,Participant's ID,str,NaN,NaN
1,Demographics,Basic_Demos-Enroll_Season,Season of enrollment,str,"Spring, Summer, Fall, Winter",NaN
2,Demographics,Basic_Demos-Age,Age of participant,float,NaN,NaN
3,Demographics,Basic_Demos-Sex,Sex of participant,categorical int,"0,1","0=Male, 1=Female"
4,Children's Global Assessment Scale,CGAS-Season,Season of participation,str,"Spring, Summer, Fall, Winter",NaN
5,Children's Global Assessment Scale,CGAS-CGAS_Score,Children's Global Assessment Scale Score,int,NaN,NaN
6,Physical Measures,Physical-Season,Season of participation,str,"Spring, Summer, Fall, Winter",NaN
7,Physical Measures,Physical-BMI,Body Mass Index (kg/m^2),float,NaN,NaN
8,Physical Measures,Physical-Height,Height (in),float,NaN,NaN
9,Physical Measures,Physical-Weight,Weight (lbs),float,NaN,NaN


In [13]:
cat_cols = []
for i in range(data_dict.shape[0]):
    col = data_dict['Field'][i]
    if col == 'id':
        continue
    if (data_dict['Type'][i] == 'str') | ('categori' in data_dict['Type'][i]):
        cat_cols.append(col)

In [14]:
ordinal_variables = ['FGC-FGC_CU_Zone', 'FGC-FGC_CU_Zone', 'FGC-FGC_GSND_Zone', 'FGC-FGC_GSD_Zone', 'FGC-FGC_PU_Zone',
                     'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR_Zone', 'FGC-FGC_TL_Zone', 'BIA-BIA_Activity_Level_num', 'BIA-BIA_Frame_num',
                     'PreInt_EduHx-computerinternet_hoursday']

ordinal_variables += [col for col in cat_cols if (('PCIAT' in col)&('Season' not in col))]
print(len(ordinal_variables))
ordinal_variables

31


['FGC-FGC_CU_Zone',
 'FGC-FGC_CU_Zone',
 'FGC-FGC_GSND_Zone',
 'FGC-FGC_GSD_Zone',
 'FGC-FGC_PU_Zone',
 'FGC-FGC_SRL_Zone',
 'FGC-FGC_SRR_Zone',
 'FGC-FGC_TL_Zone',
 'BIA-BIA_Activity_Level_num',
 'BIA-BIA_Frame_num',
 'PreInt_EduHx-computerinternet_hoursday',
 'PCIAT-PCIAT_01',
 'PCIAT-PCIAT_02',
 'PCIAT-PCIAT_03',
 'PCIAT-PCIAT_04',
 'PCIAT-PCIAT_05',
 'PCIAT-PCIAT_06',
 'PCIAT-PCIAT_07',
 'PCIAT-PCIAT_08',
 'PCIAT-PCIAT_09',
 'PCIAT-PCIAT_10',
 'PCIAT-PCIAT_11',
 'PCIAT-PCIAT_12',
 'PCIAT-PCIAT_13',
 'PCIAT-PCIAT_14',
 'PCIAT-PCIAT_15',
 'PCIAT-PCIAT_16',
 'PCIAT-PCIAT_17',
 'PCIAT-PCIAT_18',
 'PCIAT-PCIAT_19',
 'PCIAT-PCIAT_20']

In [15]:
cat_cols = list(set(cat_cols) - set(ordinal_variables))
print(len(cat_cols))
cat_cols

12


['PAQ_A-Season',
 'Basic_Demos-Enroll_Season',
 'Fitness_Endurance-Season',
 'SDS-Season',
 'Basic_Demos-Sex',
 'BIA-Season',
 'PreInt_EduHx-Season',
 'PAQ_C-Season',
 'Physical-Season',
 'PCIAT-Season',
 'CGAS-Season',
 'FGC-Season']

In [16]:
train_nonan = train_merge.dropna(subset=['sii'], ignore_index=True)
print(train_nonan.shape)
train_nonan.head()

(2736, 140)


,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,Fitness_Endurance-Season,Fitness_Endurance-Max_Stage,Fitness_Endurance-Time_Mins,Fitness_Endurance-Time_Sec,FGC-Season,FGC-FGC_CU,FGC-FGC_CU_Zone,FGC-FGC_GSND,FGC-FGC_GSND_Zone,FGC-FGC_GSD,FGC-FGC_GSD_Zone,FGC-FGC_PU,FGC-FGC_PU_Zone,FGC-FGC_SRL,FGC-FGC_SRL_Zone,FGC-FGC_SRR,FGC-FGC_SRR_Zone,FGC-FGC_TL,FGC-FGC_TL_Zone,BIA-Season,BIA-BIA_Activity_Level_num,BIA-BIA_BMC,BIA-BIA_BMI,BIA-BIA_BMR,BIA-BIA_DEE,BIA-BIA_ECW,BIA-BIA_FFM,BIA-BIA_FFMI,BIA-BIA_FMI,BIA-BIA_Fat,BIA-BIA_Frame_num,BIA-BIA_ICW,BIA-BIA_LDM,BIA-BIA_LST,BIA-BIA_SMM,BIA-BIA_TBW,PAQ_A-Season,PAQ_A-PAQ_A_Total,PAQ_C-Season,PAQ_C-PAQ_C_Total,PCIAT-Season,PCIAT-PCIAT_01,PCIAT-PCIAT_02,PCIAT-PCIAT_03,PCIAT-PCIAT_04,PCIAT-PCIAT_05,PCIAT-PCIAT_06,PCIAT-PCIAT_07,PCIAT-PCIAT_08,PCIAT-PCIAT_09,PCIAT-PCIAT_10,PCIAT-PCIAT_11,PCIAT-PCIAT_12,PCIAT-PCIAT_13,PCIAT-PCIAT_14,PCIAT-PCIAT_15,PCIAT-PCIAT_16,PCIAT-PCIAT_17,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,sii,Enc_1,Enc_2,Enc_3,Enc_4,Enc_5,Enc_6,Enc_7,Enc_8,Enc_9,Enc_10,Enc_11,Enc_13,Enc_14,Enc_17,Enc_18,Enc_19,Enc_20,Enc_22,Enc_23,Enc_24,Enc_25,Enc_26,Enc_27,Enc_28,Enc_29,Enc_30,Enc_31,Enc_32,Enc_33,Enc_34,Enc_35,Enc_36,Enc_37,Enc_38,Enc_39,Enc_40,Enc_41,Enc_42,Enc_43,Enc_44,Enc_45,Enc_46,Enc_47,Enc_48,Enc_50,Enc_51,Enc_52,Enc_53,Enc_54,Enc_55,Enc_56,Enc_57,Enc_58,Enc_60,Enc_61,Enc_62,Enc_63,Enc_64
0,00008ff9,Fall,5,0,Winter,51.0,Fall,16.877316,46.0,50.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fall,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,7.0,0.0,6.0,0.0,6.0,1.0,Fall,2.0,2.66855,16.8792,932.498,1492.00,8.25598,41.5862,13.8177,3.06143,9.21377,1.0,24.4349,8.89536,38.9177,19.5413,32.6909,NaN,NaN,NaN,NaN,Fall,5.0,4.0,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,4.0,0.0,4.0,4.0,4.0,4.0,4.0,4.0,2.0,4.0,55.0,NaN,NaN,NaN,Fall,3.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,000fd460,Summer,9,0,NaN,NaN,Fall,14.035590,48.0,46.0,22.0,75.0,70.0,122.0,NaN,NaN,NaN,NaN,Fall,3.0,0.0,NaN,NaN,NaN,NaN,5.0,0.0,11.0,1.0,11.0,1.0,3.0,0.0,Winter,2.0,2.57949,14.0371,936.656,1498.65,6.01993,42.0291,12.8254,1.21172,3.97085,1.0,21.0352,14.97400,39.4497,15.4107,27.0552,NaN,NaN,Fall,2.340,Fall,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Fall,46.0,64.0,Summer,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,00105258,Summer,10,1,Fall,71.0,Fall,16.648696,56.5,75.6,NaN,65.0,94.0,117.0,Fall,5.0,7.0,33.0,Fall,20.0,1.0,10.2,1.0,14.7,2.0,7.0,1.0,10.0,1.0,10.0,1.0,5.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Summer,2.170,Fall,5.0,2.0,2.0,1.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,2.0,2.0,1.0,1.0,28.0,Fall,38.0,54.0,Summer,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00115b9f,Winter,9,0,Fall,71.0,Summer,18.292347,56.0,81.6,NaN,60.0,97.0,117.0,Summer,6.0,9.0,37.0,Summer,18.0,1.0,NaN,NaN,NaN,NaN,5.0,0.0,7.0,0.0,7.0,0.0,7.0,1.0,Summer,3.0,3.84191,18.2943,1131.430,1923.44,15.59250,62.7757,14.0740,4.22033,18.82430,2.0,30.4041,16.77900,58.9338,26.4798,45.9966,NaN,NaN,Winter,2.451,Summer,4.0,2.0,4.0,0.0,5.0,1.0,0.0,3.0,2.0,2.0,3.0,0.0,3.0,0.0,0.0,3.0,4.0,3.0,4.0,1.0,44.0,Summer,31.0,45.0

In [17]:
train_one_hot = train_nonan.copy()
test_one_hot = test_merge.copy()

In [18]:
for col in cat_cols:
    if 'PCIAT' not in col:
        train_sr = train_one_hot[col].fillna('Missing').apply(str)        
        test_sr = test_one_hot[col].fillna('Missing').apply(str)
        train_one_hot[col] = train_sr
        test_one_hot[col] = test_sr
        concat_sr = pd.concat([train_sr, test_sr])
        unique_list = sorted(concat_sr.unique().tolist())

        for v in unique_list:
            new_col_name = col + '_' + v
            train_one_hot[new_col_name] = pd.Series(train_one_hot[col] == v).astype(int)
            test_one_hot[new_col_name] = pd.Series(test_one_hot[col] == v).astype(int)
    else:
        train_sr = train_one_hot[col].fillna('Missing').apply(str)
        train_one_hot[col] = train_sr
        unique_list = sorted(train_sr.unique().tolist())

        for v in unique_list:
            new_col_name = col + '_' + v
            train_one_hot[new_col_name] = pd.Series(train_one_hot[col] == v).astype(int)

train_one_hot.drop(cat_cols, axis=1, inplace=True)
test_one_hot.drop([col for col in cat_cols if 'PCIAT' not in col], axis=1, inplace=True)

In [19]:
print(train_one_hot.shape)
train_one_hot.head()

(2736, 183)


,id,Basic_Demos-Age,CGAS-CGAS_Score,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,Fitness_Endurance-Max_Stage,Fitness_Endurance-Time_Mins,Fitness_Endurance-Time_Sec,FGC-FGC_CU,FGC-FGC_CU_Zone,FGC-FGC_GSND,FGC-FGC_GSND_Zone,FGC-FGC_GSD,FGC-FGC_GSD_Zone,FGC-FGC_PU,FGC-FGC_PU_Zone,FGC-FGC_SRL,FGC-FGC_SRL_Zone,FGC-FGC_SRR,FGC-FGC_SRR_Zone,FGC-FGC_TL,FGC-FGC_TL_Zone,BIA-BIA_Activity_Level_num,BIA-BIA_BMC,BIA-BIA_BMI,BIA-BIA_BMR,BIA-BIA_DEE,BIA-BIA_ECW,BIA-BIA_FFM,BIA-BIA_FFMI,BIA-BIA_FMI,BIA-BIA_Fat,BIA-BIA_Frame_num,BIA-BIA_ICW,BIA-BIA_LDM,BIA-BIA_LST,BIA-BIA_SMM,BIA-BIA_TBW,PAQ_A-PAQ_A_Total,PAQ_C-PAQ_C_Total,PCIAT-PCIAT_01,PCIAT-PCIAT_02,PCIAT-PCIAT_03,PCIAT-PCIAT_04,PCIAT-PCIAT_05,PCIAT-PCIAT_06,PCIAT-PCIAT_07,PCIAT-PCIAT_08,PCIAT-PCIAT_09,PCIAT-PCIAT_10,PCIAT-PCIAT_11,PCIAT-PCIAT_12,PCIAT-PCIAT_13,PCIAT-PCIAT_14,PCIAT-PCIAT_15,PCIAT-PCIAT_16,PCIAT-PCIAT_17,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-computerinternet_hoursday,sii,Enc_1,Enc_2,Enc_3,Enc_4,Enc_5,Enc_6,Enc_7,Enc_8,Enc_9,Enc_10,Enc_11,Enc_13,Enc_14,Enc_17,Enc_18,Enc_19,Enc_20,Enc_22,Enc_23,Enc_24,Enc_25,Enc_26,Enc_27,Enc_28,Enc_29,Enc_30,Enc_31,Enc_32,Enc_33,Enc_34,Enc_35,Enc_36,Enc_37,Enc_38,Enc_39,Enc_40,Enc_41,Enc_42,Enc_43,Enc_44,Enc_45,Enc_46,Enc_47,Enc_48,Enc_50,Enc_51,Enc_52,Enc_53,Enc_54,Enc_55,Enc_56,Enc_57,Enc_58,Enc_60,Enc_61,Enc_62,Enc_63,Enc_64,PAQ_A-Season_Fall,PAQ_A-Season_Missing,PAQ_A-Season_Spring,PAQ_A-Season_Summer,PAQ_A-Season_Winter,Basic_Demos-Enroll_Season_Fall,Basic_Demos-Enroll_Season_Spring,Basic_Demos-Enroll_Season_Summer,Basic_Demos-Enroll_Season_Winter,Fitness_Endurance-Season_Fall,Fitness_Endurance-Season_Missing,Fitness_Endurance-Season_Spring,Fitness_Endurance-Season_Summer,Fitness_Endurance-Season_Winter,SDS-Season_Fall,SDS-Season_Missing,SDS-Season_Spring,SDS-Season_Summer,SDS-Season_Winter,Basic_Demos-Sex_0,Basic_Demos-Sex_1,BIA-Season_Fall,BIA-Season_Missing,BIA-Season_Spring,BIA-Season_Summer,BIA-Season_Winter,PreInt_EduHx-Season_Fall,PreInt_EduHx-Season_Missing,PreInt_EduHx-Season_Spring,PreInt_EduHx-Season_Summer,PreInt_EduHx-Season_Winter,PAQ_C-Season_Fall,PAQ_C-Season_Missing,PAQ_C-Season_Spring,PAQ_C-Season_Summer,PAQ_C-Season_Winter,Physical-Season_Fall,Physical-Season_Missing,Physical-Season_Spring,Physical-Season_Summer,Physical-Season_Winter,PCIAT-Season_Fall,PCIAT-Season_Spring,PCIAT-Season_Summer,PCIAT-Season_Winter,CGAS-Season_Fall,CGAS-Season_Missing,CGAS-Season_Spring,CGAS-Season_Summer,CGAS-Season_Winter,FGC-Season_Fall,FGC-Season_Missing,FGC-Season_Spring,FGC-Season_Summer,FGC-Season_Winter
0,00008ff9,5,51.0,16.877316,46.0,50.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,7.0,0.0,6.0,0.0,6.0,1.0,2.0,2.66855,16.8792,932.498,1492.00,8.25598,41.5862,13.8177,3.06143,9.21377,1.0,24.4349,8.89536,38.9177,19.5413,32.6909,NaN,NaN,5.0,4.0,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,4.0,0.0,4.0,4.0,4.0,4.0,4.0,4.0,2.0,4.0,55.0,NaN,NaN,3.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0
1,000fd460,9,NaN,14.035590,48.0,46.0,22.0,75.0,70.0,122.0,NaN,NaN,NaN,3.0,0.0,NaN,NaN,NaN,NaN,5.0,0.0,11.0,1.0,11.0,1.0,3.0,0.0,2.0,2.57949,14.0371,936.656,1498.65,6.01993,42.0291,12.8254,1.21172,3.97085,1.0,21.0352,14.97400,39.4497,15.4107,27.0552,NaN,2.340,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46.0,64.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0,0,

In [20]:
print(test_one_hot.shape)
test_one_hot.head()

(20, 157)


,id,Basic_Demos-Age,CGAS-CGAS_Score,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,Fitness_Endurance-Max_Stage,Fitness_Endurance-Time_Mins,Fitness_Endurance-Time_Sec,FGC-FGC_CU,FGC-FGC_CU_Zone,FGC-FGC_GSND,FGC-FGC_GSND_Zone,FGC-FGC_GSD,FGC-FGC_GSD_Zone,FGC-FGC_PU,FGC-FGC_PU_Zone,FGC-FGC_SRL,FGC-FGC_SRL_Zone,FGC-FGC_SRR,FGC-FGC_SRR_Zone,FGC-FGC_TL,FGC-FGC_TL_Zone,BIA-BIA_Activity_Level_num,BIA-BIA_BMC,BIA-BIA_BMI,BIA-BIA_BMR,BIA-BIA_DEE,BIA-BIA_ECW,BIA-BIA_FFM,BIA-BIA_FFMI,BIA-BIA_FMI,BIA-BIA_Fat,BIA-BIA_Frame_num,BIA-BIA_ICW,BIA-BIA_LDM,BIA-BIA_LST,BIA-BIA_SMM,BIA-BIA_TBW,PAQ_A-PAQ_A_Total,PAQ_C-PAQ_C_Total,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-computerinternet_hoursday,Enc_1,Enc_2,Enc_3,Enc_4,Enc_5,Enc_6,Enc_7,Enc_8,Enc_9,Enc_10,Enc_11,Enc_13,Enc_14,Enc_17,Enc_18,Enc_19,Enc_20,Enc_22,Enc_23,Enc_24,Enc_25,Enc_26,Enc_27,Enc_28,Enc_29,Enc_30,Enc_31,Enc_32,Enc_33,Enc_34,Enc_35,Enc_36,Enc_37,Enc_38,Enc_39,Enc_40,Enc_41,Enc_42,Enc_43,Enc_44,Enc_45,Enc_46,Enc_47,Enc_48,Enc_50,Enc_51,Enc_52,Enc_53,Enc_54,Enc_55,Enc_56,Enc_57,Enc_58,Enc_60,Enc_61,Enc_62,Enc_63,Enc_64,PAQ_A-Season_Fall,PAQ_A-Season_Missing,PAQ_A-Season_Spring,PAQ_A-Season_Summer,PAQ_A-Season_Winter,Basic_Demos-Enroll_Season_Fall,Basic_Demos-Enroll_Season_Spring,Basic_Demos-Enroll_Season_Summer,Basic_Demos-Enroll_Season_Winter,Fitness_Endurance-Season_Fall,Fitness_Endurance-Season_Missing,Fitness_Endurance-Season_Spring,Fitness_Endurance-Season_Summer,Fitness_Endurance-Season_Winter,SDS-Season_Fall,SDS-Season_Missing,SDS-Season_Spring,SDS-Season_Summer,SDS-Season_Winter,Basic_Demos-Sex_0,Basic_Demos-Sex_1,BIA-Season_Fall,BIA-Season_Missing,BIA-Season_Spring,BIA-Season_Summer,BIA-Season_Winter,PreInt_EduHx-Season_Fall,PreInt_EduHx-Season_Missing,PreInt_EduHx-Season_Spring,PreInt_EduHx-Season_Summer,PreInt_EduHx-Season_Winter,PAQ_C-Season_Fall,PAQ_C-Season_Missing,PAQ_C-Season_Spring,PAQ_C-Season_Summer,PAQ_C-Season_Winter,Physical-Season_Fall,Physical-Season_Missing,Physical-Season_Spring,Physical-Season_Summer,Physical-Season_Winter,CGAS-Season_Fall,CGAS-Season_Missing,CGAS-Season_Spring,CGAS-Season_Summer,CGAS-Season_Winter,FGC-Season_Fall,FGC-Season_Missing,FGC-Season_Spring,FGC-Season_Summer,FGC-Season_Winter
0,00008ff9,5,51.0,16.877316,46.0,50.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,7.0,0.0,6.0,0.0,6.0,1.0,2.0,2.66855,16.8792,932.498,1492.00,8.25598,41.5862,13.8177,3.06143,9.21377,1.0,24.4349,8.89536,38.9177,19.5413,32.6909,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0
1,000fd460,9,NaN,14.035590,48.0,46.0,22.0,75.0,70.0,122.0,NaN,NaN,NaN,3.0,0.0,NaN,NaN,NaN,NaN,5.0,0.0,11.0,1.0,11.0,1.0,3.0,0.0,2.0,2.57949,14.0371,936.656,1498.65,6.01993,42.0291,12.8254,1.21172,3.97085,1.0,21.0352,14.97400,39.4497,15.4107,27.0552,NaN,2.340,46.0,64.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0
2,00105258,10,71.0,16.648696,56.5,75.6,NaN,65.0,94.0,117.0,5.0,7.0,33.0,20.0,1.0,10.2,1.0,14.7,2.0,7.0,1.0,10.0,1.0,10.0,1.0,5.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.170,38.0,54.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,

In [21]:
train_one_hot.drop('id', axis=1, inplace=True)
test_one_hot.drop('id', axis=1, inplace=True)

In [22]:
train = train_one_hot.copy()
test = test_one_hot.copy()

In [23]:
print(train.shape)
train.head()

(2736, 182)


,Basic_Demos-Age,CGAS-CGAS_Score,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,Fitness_Endurance-Max_Stage,Fitness_Endurance-Time_Mins,Fitness_Endurance-Time_Sec,FGC-FGC_CU,FGC-FGC_CU_Zone,FGC-FGC_GSND,FGC-FGC_GSND_Zone,FGC-FGC_GSD,FGC-FGC_GSD_Zone,FGC-FGC_PU,FGC-FGC_PU_Zone,FGC-FGC_SRL,FGC-FGC_SRL_Zone,FGC-FGC_SRR,FGC-FGC_SRR_Zone,FGC-FGC_TL,FGC-FGC_TL_Zone,BIA-BIA_Activity_Level_num,BIA-BIA_BMC,BIA-BIA_BMI,BIA-BIA_BMR,BIA-BIA_DEE,BIA-BIA_ECW,BIA-BIA_FFM,BIA-BIA_FFMI,BIA-BIA_FMI,BIA-BIA_Fat,BIA-BIA_Frame_num,BIA-BIA_ICW,BIA-BIA_LDM,BIA-BIA_LST,BIA-BIA_SMM,BIA-BIA_TBW,PAQ_A-PAQ_A_Total,PAQ_C-PAQ_C_Total,PCIAT-PCIAT_01,PCIAT-PCIAT_02,PCIAT-PCIAT_03,PCIAT-PCIAT_04,PCIAT-PCIAT_05,PCIAT-PCIAT_06,PCIAT-PCIAT_07,PCIAT-PCIAT_08,PCIAT-PCIAT_09,PCIAT-PCIAT_10,PCIAT-PCIAT_11,PCIAT-PCIAT_12,PCIAT-PCIAT_13,PCIAT-PCIAT_14,PCIAT-PCIAT_15,PCIAT-PCIAT_16,PCIAT-PCIAT_17,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-computerinternet_hoursday,sii,Enc_1,Enc_2,Enc_3,Enc_4,Enc_5,Enc_6,Enc_7,Enc_8,Enc_9,Enc_10,Enc_11,Enc_13,Enc_14,Enc_17,Enc_18,Enc_19,Enc_20,Enc_22,Enc_23,Enc_24,Enc_25,Enc_26,Enc_27,Enc_28,Enc_29,Enc_30,Enc_31,Enc_32,Enc_33,Enc_34,Enc_35,Enc_36,Enc_37,Enc_38,Enc_39,Enc_40,Enc_41,Enc_42,Enc_43,Enc_44,Enc_45,Enc_46,Enc_47,Enc_48,Enc_50,Enc_51,Enc_52,Enc_53,Enc_54,Enc_55,Enc_56,Enc_57,Enc_58,Enc_60,Enc_61,Enc_62,Enc_63,Enc_64,PAQ_A-Season_Fall,PAQ_A-Season_Missing,PAQ_A-Season_Spring,PAQ_A-Season_Summer,PAQ_A-Season_Winter,Basic_Demos-Enroll_Season_Fall,Basic_Demos-Enroll_Season_Spring,Basic_Demos-Enroll_Season_Summer,Basic_Demos-Enroll_Season_Winter,Fitness_Endurance-Season_Fall,Fitness_Endurance-Season_Missing,Fitness_Endurance-Season_Spring,Fitness_Endurance-Season_Summer,Fitness_Endurance-Season_Winter,SDS-Season_Fall,SDS-Season_Missing,SDS-Season_Spring,SDS-Season_Summer,SDS-Season_Winter,Basic_Demos-Sex_0,Basic_Demos-Sex_1,BIA-Season_Fall,BIA-Season_Missing,BIA-Season_Spring,BIA-Season_Summer,BIA-Season_Winter,PreInt_EduHx-Season_Fall,PreInt_EduHx-Season_Missing,PreInt_EduHx-Season_Spring,PreInt_EduHx-Season_Summer,PreInt_EduHx-Season_Winter,PAQ_C-Season_Fall,PAQ_C-Season_Missing,PAQ_C-Season_Spring,PAQ_C-Season_Summer,PAQ_C-Season_Winter,Physical-Season_Fall,Physical-Season_Missing,Physical-Season_Spring,Physical-Season_Summer,Physical-Season_Winter,PCIAT-Season_Fall,PCIAT-Season_Spring,PCIAT-Season_Summer,PCIAT-Season_Winter,CGAS-Season_Fall,CGAS-Season_Missing,CGAS-Season_Spring,CGAS-Season_Summer,CGAS-Season_Winter,FGC-Season_Fall,FGC-Season_Missing,FGC-Season_Spring,FGC-Season_Summer,FGC-Season_Winter
0,5,51.0,16.877316,46.0,50.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,7.0,0.0,6.0,0.0,6.0,1.0,2.0,2.66855,16.8792,932.498,1492.00,8.25598,41.5862,13.8177,3.06143,9.21377,1.0,24.4349,8.89536,38.9177,19.5413,32.6909,NaN,NaN,5.0,4.0,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,4.0,0.0,4.0,4.0,4.0,4.0,4.0,4.0,2.0,4.0,55.0,NaN,NaN,3.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0
1,9,NaN,14.035590,48.0,46.0,22.0,75.0,70.0,122.0,NaN,NaN,NaN,3.0,0.0,NaN,NaN,NaN,NaN,5.0,0.0,11.0,1.0,11.0,1.0,3.0,0.0,2.0,2.57949,14.0371,936.656,1498.65,6.01993,42.0291,12.8254,1.21172,3.97085,1.0,21.0352,14.97400,39.4497,15.4107,27.0552,NaN,2.340,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46.0,64.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1

In [24]:
print(test.shape)
test.head()

(20, 156)


,Basic_Demos-Age,CGAS-CGAS_Score,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,Fitness_Endurance-Max_Stage,Fitness_Endurance-Time_Mins,Fitness_Endurance-Time_Sec,FGC-FGC_CU,FGC-FGC_CU_Zone,FGC-FGC_GSND,FGC-FGC_GSND_Zone,FGC-FGC_GSD,FGC-FGC_GSD_Zone,FGC-FGC_PU,FGC-FGC_PU_Zone,FGC-FGC_SRL,FGC-FGC_SRL_Zone,FGC-FGC_SRR,FGC-FGC_SRR_Zone,FGC-FGC_TL,FGC-FGC_TL_Zone,BIA-BIA_Activity_Level_num,BIA-BIA_BMC,BIA-BIA_BMI,BIA-BIA_BMR,BIA-BIA_DEE,BIA-BIA_ECW,BIA-BIA_FFM,BIA-BIA_FFMI,BIA-BIA_FMI,BIA-BIA_Fat,BIA-BIA_Frame_num,BIA-BIA_ICW,BIA-BIA_LDM,BIA-BIA_LST,BIA-BIA_SMM,BIA-BIA_TBW,PAQ_A-PAQ_A_Total,PAQ_C-PAQ_C_Total,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-computerinternet_hoursday,Enc_1,Enc_2,Enc_3,Enc_4,Enc_5,Enc_6,Enc_7,Enc_8,Enc_9,Enc_10,Enc_11,Enc_13,Enc_14,Enc_17,Enc_18,Enc_19,Enc_20,Enc_22,Enc_23,Enc_24,Enc_25,Enc_26,Enc_27,Enc_28,Enc_29,Enc_30,Enc_31,Enc_32,Enc_33,Enc_34,Enc_35,Enc_36,Enc_37,Enc_38,Enc_39,Enc_40,Enc_41,Enc_42,Enc_43,Enc_44,Enc_45,Enc_46,Enc_47,Enc_48,Enc_50,Enc_51,Enc_52,Enc_53,Enc_54,Enc_55,Enc_56,Enc_57,Enc_58,Enc_60,Enc_61,Enc_62,Enc_63,Enc_64,PAQ_A-Season_Fall,PAQ_A-Season_Missing,PAQ_A-Season_Spring,PAQ_A-Season_Summer,PAQ_A-Season_Winter,Basic_Demos-Enroll_Season_Fall,Basic_Demos-Enroll_Season_Spring,Basic_Demos-Enroll_Season_Summer,Basic_Demos-Enroll_Season_Winter,Fitness_Endurance-Season_Fall,Fitness_Endurance-Season_Missing,Fitness_Endurance-Season_Spring,Fitness_Endurance-Season_Summer,Fitness_Endurance-Season_Winter,SDS-Season_Fall,SDS-Season_Missing,SDS-Season_Spring,SDS-Season_Summer,SDS-Season_Winter,Basic_Demos-Sex_0,Basic_Demos-Sex_1,BIA-Season_Fall,BIA-Season_Missing,BIA-Season_Spring,BIA-Season_Summer,BIA-Season_Winter,PreInt_EduHx-Season_Fall,PreInt_EduHx-Season_Missing,PreInt_EduHx-Season_Spring,PreInt_EduHx-Season_Summer,PreInt_EduHx-Season_Winter,PAQ_C-Season_Fall,PAQ_C-Season_Missing,PAQ_C-Season_Spring,PAQ_C-Season_Summer,PAQ_C-Season_Winter,Physical-Season_Fall,Physical-Season_Missing,Physical-Season_Spring,Physical-Season_Summer,Physical-Season_Winter,CGAS-Season_Fall,CGAS-Season_Missing,CGAS-Season_Spring,CGAS-Season_Summer,CGAS-Season_Winter,FGC-Season_Fall,FGC-Season_Missing,FGC-Season_Spring,FGC-Season_Summer,FGC-Season_Winter
0,5,51.0,16.877316,46.0,50.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,7.0,0.0,6.0,0.0,6.0,1.0,2.0,2.66855,16.8792,932.498,1492.00,8.25598,41.5862,13.8177,3.06143,9.21377,1.0,24.4349,8.89536,38.9177,19.5413,32.6909,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0
1,9,NaN,14.035590,48.0,46.0,22.0,75.0,70.0,122.0,NaN,NaN,NaN,3.0,0.0,NaN,NaN,NaN,NaN,5.0,0.0,11.0,1.0,11.0,1.0,3.0,0.0,2.0,2.57949,14.0371,936.656,1498.65,6.01993,42.0291,12.8254,1.21172,3.97085,1.0,21.0352,14.97400,39.4497,15.4107,27.0552,NaN,2.340,46.0,64.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0
2,10,71.0,16.648696,56.5,75.6,NaN,65.0,94.0,117.0,5.0,7.0,33.0,20.0,1.0,10.2,1.0,14.7,2.0,7.0,1.0,10.0,1.0,10.0,1.0,5.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.170,38.0,54.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,

In [25]:
imputer = KNNImputer(n_neighbors=5)
imputed_data = imputer.fit_transform(train)

In [26]:
train_imputed = pd.DataFrame(imputed_data, columns=train.columns)
print(train_imputed.shape)
train_imputed.head()

(2736, 182)


,Basic_Demos-Age,CGAS-CGAS_Score,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,Fitness_Endurance-Max_Stage,Fitness_Endurance-Time_Mins,Fitness_Endurance-Time_Sec,FGC-FGC_CU,FGC-FGC_CU_Zone,FGC-FGC_GSND,FGC-FGC_GSND_Zone,FGC-FGC_GSD,FGC-FGC_GSD_Zone,FGC-FGC_PU,FGC-FGC_PU_Zone,FGC-FGC_SRL,FGC-FGC_SRL_Zone,FGC-FGC_SRR,FGC-FGC_SRR_Zone,FGC-FGC_TL,FGC-FGC_TL_Zone,BIA-BIA_Activity_Level_num,BIA-BIA_BMC,BIA-BIA_BMI,BIA-BIA_BMR,BIA-BIA_DEE,BIA-BIA_ECW,BIA-BIA_FFM,BIA-BIA_FFMI,BIA-BIA_FMI,BIA-BIA_Fat,BIA-BIA_Frame_num,BIA-BIA_ICW,BIA-BIA_LDM,BIA-BIA_LST,BIA-BIA_SMM,BIA-BIA_TBW,PAQ_A-PAQ_A_Total,PAQ_C-PAQ_C_Total,PCIAT-PCIAT_01,PCIAT-PCIAT_02,PCIAT-PCIAT_03,PCIAT-PCIAT_04,PCIAT-PCIAT_05,PCIAT-PCIAT_06,PCIAT-PCIAT_07,PCIAT-PCIAT_08,PCIAT-PCIAT_09,PCIAT-PCIAT_10,PCIAT-PCIAT_11,PCIAT-PCIAT_12,PCIAT-PCIAT_13,PCIAT-PCIAT_14,PCIAT-PCIAT_15,PCIAT-PCIAT_16,PCIAT-PCIAT_17,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-computerinternet_hoursday,sii,Enc_1,Enc_2,Enc_3,Enc_4,Enc_5,Enc_6,Enc_7,Enc_8,Enc_9,Enc_10,Enc_11,Enc_13,Enc_14,Enc_17,Enc_18,Enc_19,Enc_20,Enc_22,Enc_23,Enc_24,Enc_25,Enc_26,Enc_27,Enc_28,Enc_29,Enc_30,Enc_31,Enc_32,Enc_33,Enc_34,Enc_35,Enc_36,Enc_37,Enc_38,Enc_39,Enc_40,Enc_41,Enc_42,Enc_43,Enc_44,Enc_45,Enc_46,Enc_47,Enc_48,Enc_50,Enc_51,Enc_52,Enc_53,Enc_54,Enc_55,Enc_56,Enc_57,Enc_58,Enc_60,Enc_61,Enc_62,Enc_63,Enc_64,PAQ_A-Season_Fall,PAQ_A-Season_Missing,PAQ_A-Season_Spring,PAQ_A-Season_Summer,PAQ_A-Season_Winter,Basic_Demos-Enroll_Season_Fall,Basic_Demos-Enroll_Season_Spring,Basic_Demos-Enroll_Season_Summer,Basic_Demos-Enroll_Season_Winter,Fitness_Endurance-Season_Fall,Fitness_Endurance-Season_Missing,Fitness_Endurance-Season_Spring,Fitness_Endurance-Season_Summer,Fitness_Endurance-Season_Winter,SDS-Season_Fall,SDS-Season_Missing,SDS-Season_Spring,SDS-Season_Summer,SDS-Season_Winter,Basic_Demos-Sex_0,Basic_Demos-Sex_1,BIA-Season_Fall,BIA-Season_Missing,BIA-Season_Spring,BIA-Season_Summer,BIA-Season_Winter,PreInt_EduHx-Season_Fall,PreInt_EduHx-Season_Missing,PreInt_EduHx-Season_Spring,PreInt_EduHx-Season_Summer,PreInt_EduHx-Season_Winter,PAQ_C-Season_Fall,PAQ_C-Season_Missing,PAQ_C-Season_Spring,PAQ_C-Season_Summer,PAQ_C-Season_Winter,Physical-Season_Fall,Physical-Season_Missing,Physical-Season_Spring,Physical-Season_Summer,Physical-Season_Winter,PCIAT-Season_Fall,PCIAT-Season_Spring,PCIAT-Season_Summer,PCIAT-Season_Winter,CGAS-Season_Fall,CGAS-Season_Missing,CGAS-Season_Spring,CGAS-Season_Summer,CGAS-Season_Winter,FGC-Season_Fall,FGC-Season_Missing,FGC-Season_Spring,FGC-Season_Summer,FGC-Season_Winter
0,5.0,51.0,16.877316,46.0,50.8,24.2,66.8,79.4,109.6,4.6,7.4,16.8,0.0,0.0,14.44,1.2,15.06,1.4,0.0,0.0,7.0,0.0,6.0,0.0,6.0,1.0,2.0,2.668550,16.8792,932.498,1492.00,8.25598,41.58620,13.81770,3.061430,9.21377,1.0,24.43490,8.89536,38.91770,19.5413,32.69090,1.912,2.170,5.0,4.0,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,4.0,0.0,4.0,4.0,4.0,4.0,4.0,4.0,2.0,4.0,55.0,42.4,59.4,3.0,2.0,1.940226,7.090114,6.851843,3.312979,3.731173,4.773416,6.047329,0.303090,1.557901,0.172479,2.217920,4.374257,2.017492,7.408865,3.042647,3.437833,0.000000,8.375595,4.072908,2.629780,5.833642,2.703716,2.645724,8.372981,7.545446,7.182353,1.780870,5.257917,3.777040,5.550829,2.086145,4.704612,1.021409,4.335701,0.973353,7.225902,6.248177,5.732063,1.432999,0.448819,4.071324,3.883944,2.190738,4.255717,5.636615,1.672150,5.847755,8.796700,6.136100,4.010981,2.648712,6.566596,3.705575,2.125699,7.886980,0.242161,7.096453,4.221281,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
1,9.0,63.4,14.035590,48.0,46.0,22.0,75.0,70.0,122.0,5.4,8.4,19.2,3.0,0.0,21.82,1.8,24.44,2.0,5.0,0.0,11.0,1.0,11.0,1.0,3.0,0.0,2.0,2.579490,14.0371,936.656,1498.65,6.01993,42.02910,12.8254

In [27]:
concat_df = pd.concat([train_imputed[test.columns], test], ignore_index=True)
print(concat_df.shape)
concat_df.head()

(2756, 156)


,Basic_Demos-Age,CGAS-CGAS_Score,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,Fitness_Endurance-Max_Stage,Fitness_Endurance-Time_Mins,Fitness_Endurance-Time_Sec,FGC-FGC_CU,FGC-FGC_CU_Zone,FGC-FGC_GSND,FGC-FGC_GSND_Zone,FGC-FGC_GSD,FGC-FGC_GSD_Zone,FGC-FGC_PU,FGC-FGC_PU_Zone,FGC-FGC_SRL,FGC-FGC_SRL_Zone,FGC-FGC_SRR,FGC-FGC_SRR_Zone,FGC-FGC_TL,FGC-FGC_TL_Zone,BIA-BIA_Activity_Level_num,BIA-BIA_BMC,BIA-BIA_BMI,BIA-BIA_BMR,BIA-BIA_DEE,BIA-BIA_ECW,BIA-BIA_FFM,BIA-BIA_FFMI,BIA-BIA_FMI,BIA-BIA_Fat,BIA-BIA_Frame_num,BIA-BIA_ICW,BIA-BIA_LDM,BIA-BIA_LST,BIA-BIA_SMM,BIA-BIA_TBW,PAQ_A-PAQ_A_Total,PAQ_C-PAQ_C_Total,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-computerinternet_hoursday,Enc_1,Enc_2,Enc_3,Enc_4,Enc_5,Enc_6,Enc_7,Enc_8,Enc_9,Enc_10,Enc_11,Enc_13,Enc_14,Enc_17,Enc_18,Enc_19,Enc_20,Enc_22,Enc_23,Enc_24,Enc_25,Enc_26,Enc_27,Enc_28,Enc_29,Enc_30,Enc_31,Enc_32,Enc_33,Enc_34,Enc_35,Enc_36,Enc_37,Enc_38,Enc_39,Enc_40,Enc_41,Enc_42,Enc_43,Enc_44,Enc_45,Enc_46,Enc_47,Enc_48,Enc_50,Enc_51,Enc_52,Enc_53,Enc_54,Enc_55,Enc_56,Enc_57,Enc_58,Enc_60,Enc_61,Enc_62,Enc_63,Enc_64,PAQ_A-Season_Fall,PAQ_A-Season_Missing,PAQ_A-Season_Spring,PAQ_A-Season_Summer,PAQ_A-Season_Winter,Basic_Demos-Enroll_Season_Fall,Basic_Demos-Enroll_Season_Spring,Basic_Demos-Enroll_Season_Summer,Basic_Demos-Enroll_Season_Winter,Fitness_Endurance-Season_Fall,Fitness_Endurance-Season_Missing,Fitness_Endurance-Season_Spring,Fitness_Endurance-Season_Summer,Fitness_Endurance-Season_Winter,SDS-Season_Fall,SDS-Season_Missing,SDS-Season_Spring,SDS-Season_Summer,SDS-Season_Winter,Basic_Demos-Sex_0,Basic_Demos-Sex_1,BIA-Season_Fall,BIA-Season_Missing,BIA-Season_Spring,BIA-Season_Summer,BIA-Season_Winter,PreInt_EduHx-Season_Fall,PreInt_EduHx-Season_Missing,PreInt_EduHx-Season_Spring,PreInt_EduHx-Season_Summer,PreInt_EduHx-Season_Winter,PAQ_C-Season_Fall,PAQ_C-Season_Missing,PAQ_C-Season_Spring,PAQ_C-Season_Summer,PAQ_C-Season_Winter,Physical-Season_Fall,Physical-Season_Missing,Physical-Season_Spring,Physical-Season_Summer,Physical-Season_Winter,CGAS-Season_Fall,CGAS-Season_Missing,CGAS-Season_Spring,CGAS-Season_Summer,CGAS-Season_Winter,FGC-Season_Fall,FGC-Season_Missing,FGC-Season_Spring,FGC-Season_Summer,FGC-Season_Winter
0,5.0,51.0,16.877316,46.0,50.8,24.2,66.8,79.4,109.6,4.6,7.4,16.8,0.0,0.0,14.44,1.2,15.06,1.4,0.0,0.0,7.0,0.0,6.0,0.0,6.0,1.0,2.0,2.668550,16.8792,932.498,1492.00,8.25598,41.58620,13.81770,3.061430,9.21377,1.0,24.43490,8.89536,38.91770,19.5413,32.69090,1.912,2.170,42.4,59.4,3.0,1.940226,7.090114,6.851843,3.312979,3.731173,4.773416,6.047329,0.303090,1.557901,0.172479,2.217920,4.374257,2.017492,7.408865,3.042647,3.437833,0.000000,8.375595,4.072908,2.629780,5.833642,2.703716,2.645724,8.372981,7.545446,7.182353,1.780870,5.257917,3.777040,5.550829,2.086145,4.704612,1.021409,4.335701,0.973353,7.225902,6.248177,5.732063,1.432999,0.448819,4.071324,3.883944,2.190738,4.255717,5.636615,1.672150,5.847755,8.796700,6.136100,4.010981,2.648712,6.566596,3.705575,2.125699,7.886980,0.242161,7.096453,4.221281,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
1,9.0,63.4,14.035590,48.0,46.0,22.0,75.0,70.0,122.0,5.4,8.4,19.2,3.0,0.0,21.82,1.8,24.44,2.0,5.0,0.0,11.0,1.0,11.0,1.0,3.0,0.0,2.0,2.579490,14.0371,936.656,1498.65,6.01993,42.02910,12.82540,1.211720,3.97085,1.0,21.03520,14.97400,39.44970,15.4107,27.05520,1.582,2.340,46.0,64.0,0.0,2.478349,3.097783,4.403833,4.813544,3.082445,6.671233,4.776706,0.914687,0.789561,3.136578,2.443026,0.599640,2.296704,2.812380,3.759177,5.713455,3.735555,10.534909,2.719555,4.080249,2.915055,1.408422,3.000531,3.056170,5.505508,11.111015,1.010977,8.588005,3.481129,5.859673,3.365819,4.471748,3.311380,2.248477,4.994781,2.511955,1.504652,3.721809,3.318429,0.849524,2.020037,2.544952,1.289199,6.449781,2.522538,6.889

In [28]:
imputer = KNNImputer(n_neighbors=5)
imputed_data_concat = imputer.fit_transform(concat_df)

In [29]:
concat_imputed = pd.DataFrame(imputed_data_concat, columns=concat_df.columns)
print(concat_imputed.shape)
concat_imputed.head()

(2756, 156)


,Basic_Demos-Age,CGAS-CGAS_Score,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,Fitness_Endurance-Max_Stage,Fitness_Endurance-Time_Mins,Fitness_Endurance-Time_Sec,FGC-FGC_CU,FGC-FGC_CU_Zone,FGC-FGC_GSND,FGC-FGC_GSND_Zone,FGC-FGC_GSD,FGC-FGC_GSD_Zone,FGC-FGC_PU,FGC-FGC_PU_Zone,FGC-FGC_SRL,FGC-FGC_SRL_Zone,FGC-FGC_SRR,FGC-FGC_SRR_Zone,FGC-FGC_TL,FGC-FGC_TL_Zone,BIA-BIA_Activity_Level_num,BIA-BIA_BMC,BIA-BIA_BMI,BIA-BIA_BMR,BIA-BIA_DEE,BIA-BIA_ECW,BIA-BIA_FFM,BIA-BIA_FFMI,BIA-BIA_FMI,BIA-BIA_Fat,BIA-BIA_Frame_num,BIA-BIA_ICW,BIA-BIA_LDM,BIA-BIA_LST,BIA-BIA_SMM,BIA-BIA_TBW,PAQ_A-PAQ_A_Total,PAQ_C-PAQ_C_Total,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-computerinternet_hoursday,Enc_1,Enc_2,Enc_3,Enc_4,Enc_5,Enc_6,Enc_7,Enc_8,Enc_9,Enc_10,Enc_11,Enc_13,Enc_14,Enc_17,Enc_18,Enc_19,Enc_20,Enc_22,Enc_23,Enc_24,Enc_25,Enc_26,Enc_27,Enc_28,Enc_29,Enc_30,Enc_31,Enc_32,Enc_33,Enc_34,Enc_35,Enc_36,Enc_37,Enc_38,Enc_39,Enc_40,Enc_41,Enc_42,Enc_43,Enc_44,Enc_45,Enc_46,Enc_47,Enc_48,Enc_50,Enc_51,Enc_52,Enc_53,Enc_54,Enc_55,Enc_56,Enc_57,Enc_58,Enc_60,Enc_61,Enc_62,Enc_63,Enc_64,PAQ_A-Season_Fall,PAQ_A-Season_Missing,PAQ_A-Season_Spring,PAQ_A-Season_Summer,PAQ_A-Season_Winter,Basic_Demos-Enroll_Season_Fall,Basic_Demos-Enroll_Season_Spring,Basic_Demos-Enroll_Season_Summer,Basic_Demos-Enroll_Season_Winter,Fitness_Endurance-Season_Fall,Fitness_Endurance-Season_Missing,Fitness_Endurance-Season_Spring,Fitness_Endurance-Season_Summer,Fitness_Endurance-Season_Winter,SDS-Season_Fall,SDS-Season_Missing,SDS-Season_Spring,SDS-Season_Summer,SDS-Season_Winter,Basic_Demos-Sex_0,Basic_Demos-Sex_1,BIA-Season_Fall,BIA-Season_Missing,BIA-Season_Spring,BIA-Season_Summer,BIA-Season_Winter,PreInt_EduHx-Season_Fall,PreInt_EduHx-Season_Missing,PreInt_EduHx-Season_Spring,PreInt_EduHx-Season_Summer,PreInt_EduHx-Season_Winter,PAQ_C-Season_Fall,PAQ_C-Season_Missing,PAQ_C-Season_Spring,PAQ_C-Season_Summer,PAQ_C-Season_Winter,Physical-Season_Fall,Physical-Season_Missing,Physical-Season_Spring,Physical-Season_Summer,Physical-Season_Winter,CGAS-Season_Fall,CGAS-Season_Missing,CGAS-Season_Spring,CGAS-Season_Summer,CGAS-Season_Winter,FGC-Season_Fall,FGC-Season_Missing,FGC-Season_Spring,FGC-Season_Summer,FGC-Season_Winter
0,5.0,51.0,16.877316,46.0,50.8,24.2,66.8,79.4,109.6,4.6,7.4,16.8,0.0,0.0,14.44,1.2,15.06,1.4,0.0,0.0,7.0,0.0,6.0,0.0,6.0,1.0,2.0,2.668550,16.8792,932.498,1492.00,8.25598,41.58620,13.81770,3.061430,9.21377,1.0,24.43490,8.89536,38.91770,19.5413,32.69090,1.912,2.170,42.4,59.4,3.0,1.940226,7.090114,6.851843,3.312979,3.731173,4.773416,6.047329,0.303090,1.557901,0.172479,2.217920,4.374257,2.017492,7.408865,3.042647,3.437833,0.000000,8.375595,4.072908,2.629780,5.833642,2.703716,2.645724,8.372981,7.545446,7.182353,1.780870,5.257917,3.777040,5.550829,2.086145,4.704612,1.021409,4.335701,0.973353,7.225902,6.248177,5.732063,1.432999,0.448819,4.071324,3.883944,2.190738,4.255717,5.636615,1.672150,5.847755,8.796700,6.136100,4.010981,2.648712,6.566596,3.705575,2.125699,7.886980,0.242161,7.096453,4.221281,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
1,9.0,63.4,14.035590,48.0,46.0,22.0,75.0,70.0,122.0,5.4,8.4,19.2,3.0,0.0,21.82,1.8,24.44,2.0,5.0,0.0,11.0,1.0,11.0,1.0,3.0,0.0,2.0,2.579490,14.0371,936.656,1498.65,6.01993,42.02910,12.82540,1.211720,3.97085,1.0,21.03520,14.97400,39.44970,15.4107,27.05520,1.582,2.340,46.0,64.0,0.0,2.478349,3.097783,4.403833,4.813544,3.082445,6.671233,4.776706,0.914687,0.789561,3.136578,2.443026,0.599640,2.296704,2.812380,3.759177,5.713455,3.735555,10.534909,2.719555,4.080249,2.915055,1.408422,3.000531,3.056170,5.505508,11.111015,1.010977,8.588005,3.481129,5.859673,3.365819,4.471748,3.311380,2.248477,4.994781,2.511955,1.504652,3.721809,3.318429,0.849524,2.020037,2.544952,1.289199,6.449781,2.522538,6.889

In [30]:
concat_imputed.isna().sum().sum()

0

In [31]:
test_imputed = concat_imputed.tail(test.shape[0])
print(test_imputed.shape)
test_imputed.head()

(20, 156)


,Basic_Demos-Age,CGAS-CGAS_Score,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,Fitness_Endurance-Max_Stage,Fitness_Endurance-Time_Mins,Fitness_Endurance-Time_Sec,FGC-FGC_CU,FGC-FGC_CU_Zone,FGC-FGC_GSND,FGC-FGC_GSND_Zone,FGC-FGC_GSD,FGC-FGC_GSD_Zone,FGC-FGC_PU,FGC-FGC_PU_Zone,FGC-FGC_SRL,FGC-FGC_SRL_Zone,FGC-FGC_SRR,FGC-FGC_SRR_Zone,FGC-FGC_TL,FGC-FGC_TL_Zone,BIA-BIA_Activity_Level_num,BIA-BIA_BMC,BIA-BIA_BMI,BIA-BIA_BMR,BIA-BIA_DEE,BIA-BIA_ECW,BIA-BIA_FFM,BIA-BIA_FFMI,BIA-BIA_FMI,BIA-BIA_Fat,BIA-BIA_Frame_num,BIA-BIA_ICW,BIA-BIA_LDM,BIA-BIA_LST,BIA-BIA_SMM,BIA-BIA_TBW,PAQ_A-PAQ_A_Total,PAQ_C-PAQ_C_Total,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-computerinternet_hoursday,Enc_1,Enc_2,Enc_3,Enc_4,Enc_5,Enc_6,Enc_7,Enc_8,Enc_9,Enc_10,Enc_11,Enc_13,Enc_14,Enc_17,Enc_18,Enc_19,Enc_20,Enc_22,Enc_23,Enc_24,Enc_25,Enc_26,Enc_27,Enc_28,Enc_29,Enc_30,Enc_31,Enc_32,Enc_33,Enc_34,Enc_35,Enc_36,Enc_37,Enc_38,Enc_39,Enc_40,Enc_41,Enc_42,Enc_43,Enc_44,Enc_45,Enc_46,Enc_47,Enc_48,Enc_50,Enc_51,Enc_52,Enc_53,Enc_54,Enc_55,Enc_56,Enc_57,Enc_58,Enc_60,Enc_61,Enc_62,Enc_63,Enc_64,PAQ_A-Season_Fall,PAQ_A-Season_Missing,PAQ_A-Season_Spring,PAQ_A-Season_Summer,PAQ_A-Season_Winter,Basic_Demos-Enroll_Season_Fall,Basic_Demos-Enroll_Season_Spring,Basic_Demos-Enroll_Season_Summer,Basic_Demos-Enroll_Season_Winter,Fitness_Endurance-Season_Fall,Fitness_Endurance-Season_Missing,Fitness_Endurance-Season_Spring,Fitness_Endurance-Season_Summer,Fitness_Endurance-Season_Winter,SDS-Season_Fall,SDS-Season_Missing,SDS-Season_Spring,SDS-Season_Summer,SDS-Season_Winter,Basic_Demos-Sex_0,Basic_Demos-Sex_1,BIA-Season_Fall,BIA-Season_Missing,BIA-Season_Spring,BIA-Season_Summer,BIA-Season_Winter,PreInt_EduHx-Season_Fall,PreInt_EduHx-Season_Missing,PreInt_EduHx-Season_Spring,PreInt_EduHx-Season_Summer,PreInt_EduHx-Season_Winter,PAQ_C-Season_Fall,PAQ_C-Season_Missing,PAQ_C-Season_Spring,PAQ_C-Season_Summer,PAQ_C-Season_Winter,Physical-Season_Fall,Physical-Season_Missing,Physical-Season_Spring,Physical-Season_Summer,Physical-Season_Winter,CGAS-Season_Fall,CGAS-Season_Missing,CGAS-Season_Spring,CGAS-Season_Summer,CGAS-Season_Winter,FGC-Season_Fall,FGC-Season_Missing,FGC-Season_Spring,FGC-Season_Summer,FGC-Season_Winter
2736,5.0,51.00,16.877316,46.0,50.800,23.64,71.24,78.60,121.52,4.44,6.56,26.52,0.00,0.00,15.900,1.60,16.744,1.68,0.00,0.00,7.000,0.00,6.00,0.00,6.00,1.0,2.00,2.668550,16.879200,932.4980,1492.0000,8.255980,41.586200,13.817700,3.061430,9.213770,1.00,24.434900,8.895360,38.917700,19.541300,32.690900,1.75440,2.5612,39.88,56.28,3.0,4.146095,5.098936,5.751642,5.369324,1.907452,7.346219,3.697630,0.072829,2.082264,3.762738,1.974344,3.015886,4.338364,4.126766,3.388199,6.420395,5.401684,10.703547,3.947854,5.171425,4.675375,2.711288,4.898180,4.027126,7.029704,10.801527,3.229619,11.602550,3.051257,8.731516,3.972671,3.518056,3.558551,2.442623,3.994294,4.141173,4.263391,5.596790,4.488864,0.523367,2.594709,2.421323,1.695968,6.214463,8.031522,5.475738,3.961531,7.407165,10.539744,2.188900,3.814482,9.613784,4.796636,3.670116,4.546722,4.561527,2.203451,4.299928,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
2737,9.0,58.48,14.035590,48.0,46.000,22.00,75.00,70.00,122.00,5.04,7.60,27.40,3.00,0.00,16.764,1.64,17.708,1.72,5.00,0.00,11.000,1.00,11.00,1.00,3.00,0.0,2.00,2.579490,14.037100,936.6560,1498.6500,6.019930,42.029100,12.825400,1.211720,3.970850,1.00,21.035200,14.974000,39.449700,15.410700,27.055200,1.54720,2.3400,46.00,64.00,0.0,6.047417,4.569628,6.769226,6.642010,1.581988,9.567016,3.082579,0.243555,0.469492,4.061829,2.296046,2.881299,8.026555,3.502752,5.098723,6.948600,5.925978,9.315073,6.598071,5.210968,7.010351,1.336011,4.197155,3.610245,11.517286,14.203370,5.273009,10.004241,2.570210,10.669809,3.018791,2.752680,5.166960,4.326519,

In [32]:
train_imputed = feature_engineering(train_imputed)
test_imputed = feature_engineering(test_imputed)

In [33]:
if np.any(np.isinf(train_imputed)):
    print('inf in train data')
    train_imputed = train_imputed.replace([np.inf, -np.inf], np.nan)
if np.any(np.isinf(test_imputed)):
    print('inf in test data')
    test_imputed = test_imputed.replace([np.inf, -np.inf], np.nan)

inf in train data


In [34]:
imputer = KNNImputer(n_neighbors=5)
imputed_data = imputer.fit_transform(train_imputed)

In [35]:
train_imputed = pd.DataFrame(imputed_data, columns=train_imputed.columns)
print(train_imputed.shape)
train_imputed.head()

(2736, 197)


,Basic_Demos-Age,CGAS-CGAS_Score,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,Fitness_Endurance-Max_Stage,Fitness_Endurance-Time_Mins,Fitness_Endurance-Time_Sec,FGC-FGC_CU,FGC-FGC_CU_Zone,FGC-FGC_GSND,FGC-FGC_GSND_Zone,FGC-FGC_GSD,FGC-FGC_GSD_Zone,FGC-FGC_PU,FGC-FGC_PU_Zone,FGC-FGC_SRL,FGC-FGC_SRL_Zone,FGC-FGC_SRR,FGC-FGC_SRR_Zone,FGC-FGC_TL,FGC-FGC_TL_Zone,BIA-BIA_Activity_Level_num,BIA-BIA_BMC,BIA-BIA_BMI,BIA-BIA_BMR,BIA-BIA_DEE,BIA-BIA_ECW,BIA-BIA_FFM,BIA-BIA_FFMI,BIA-BIA_FMI,BIA-BIA_Fat,BIA-BIA_Frame_num,BIA-BIA_ICW,BIA-BIA_LDM,BIA-BIA_LST,BIA-BIA_SMM,BIA-BIA_TBW,PAQ_A-PAQ_A_Total,PAQ_C-PAQ_C_Total,PCIAT-PCIAT_01,PCIAT-PCIAT_02,PCIAT-PCIAT_03,PCIAT-PCIAT_04,PCIAT-PCIAT_05,PCIAT-PCIAT_06,PCIAT-PCIAT_07,PCIAT-PCIAT_08,PCIAT-PCIAT_09,PCIAT-PCIAT_10,PCIAT-PCIAT_11,PCIAT-PCIAT_12,PCIAT-PCIAT_13,PCIAT-PCIAT_14,PCIAT-PCIAT_15,PCIAT-PCIAT_16,PCIAT-PCIAT_17,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-computerinternet_hoursday,sii,Enc_1,Enc_2,Enc_3,Enc_4,Enc_5,Enc_6,Enc_7,Enc_8,Enc_9,Enc_10,Enc_11,Enc_13,Enc_14,Enc_17,Enc_18,Enc_19,Enc_20,Enc_22,Enc_23,Enc_24,Enc_25,Enc_26,Enc_27,Enc_28,Enc_29,Enc_30,Enc_31,Enc_32,Enc_33,Enc_34,Enc_35,Enc_36,Enc_37,Enc_38,Enc_39,Enc_40,Enc_41,Enc_42,Enc_43,Enc_44,Enc_45,Enc_46,Enc_47,Enc_48,Enc_50,Enc_51,Enc_52,Enc_53,Enc_54,Enc_55,Enc_56,Enc_57,Enc_58,Enc_60,Enc_61,Enc_62,Enc_63,Enc_64,PAQ_A-Season_Fall,PAQ_A-Season_Missing,PAQ_A-Season_Spring,PAQ_A-Season_Summer,PAQ_A-Season_Winter,Basic_Demos-Enroll_Season_Fall,Basic_Demos-Enroll_Season_Spring,Basic_Demos-Enroll_Season_Summer,Basic_Demos-Enroll_Season_Winter,Fitness_Endurance-Season_Fall,Fitness_Endurance-Season_Missing,Fitness_Endurance-Season_Spring,Fitness_Endurance-Season_Summer,Fitness_Endurance-Season_Winter,SDS-Season_Fall,SDS-Season_Missing,SDS-Season_Spring,SDS-Season_Summer,SDS-Season_Winter,Basic_Demos-Sex_0,Basic_Demos-Sex_1,BIA-Season_Fall,BIA-Season_Missing,BIA-Season_Spring,BIA-Season_Summer,BIA-Season_Winter,PreInt_EduHx-Season_Fall,PreInt_EduHx-Season_Missing,PreInt_EduHx-Season_Spring,PreInt_EduHx-Season_Summer,PreInt_EduHx-Season_Winter,PAQ_C-Season_Fall,PAQ_C-Season_Missing,PAQ_C-Season_Spring,PAQ_C-Season_Summer,PAQ_C-Season_Winter,Physical-Season_Fall,Physical-Season_Missing,Physical-Season_Spring,Physical-Season_Summer,Physical-Season_Winter,PCIAT-Season_Fall,PCIAT-Season_Spring,PCIAT-Season_Summer,PCIAT-Season_Winter,CGAS-Season_Fall,CGAS-Season_Missing,CGAS-Season_Spring,CGAS-Season_Summer,CGAS-Season_Winter,FGC-Season_Fall,FGC-Season_Missing,FGC-Season_Spring,FGC-Season_Summer,FGC-Season_Winter,BMI_Age,Internet_Hours_Age,BMI_Internet_Hours,BFP_BMI,FFMI_BFP,FMI_BFP,LST_TBW,BFP_BMR,BFP_DEE,BMR_Weight,DEE_Weight,SMM_Height,Muscle_to_Fat,Hydration_Status,ICW_TBW
0,5.0,51.0,16.877316,46.0,50.8,24.2,66.8,79.4,109.6,4.6,7.4,16.8,0.0,0.0,14.44,1.2,15.06,1.4,0.0,0.0,7.0,0.0,6.0,0.0,6.0,1.0,2.0,2.668550,16.8792,932.498,1492.00,8.25598,41.58620,13.81770,3.061430,9.21377,1.0,24.43490,8.89536,38.91770,19.5413,32.69090,1.912,2.170,5.0,4.0,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,4.0,0.0,4.0,4.0,4.0,4.0,4.0,4.0,2.0,4.0,55.0,42.4,59.4,3.0,2.0,1.940226,7.090114,6.851843,3.312979,3.731173,4.773416,6.047329,0.303090,1.557901,0.172479,2.217920,4.374257,2.017492,7.408865,3.042647,3.437833,0.000000,8.375595,4.072908,2.629780,5.833642,2.703716,2.645724,8.372981,7.545446,7.182353,1.780870,5.257917,3.777040,5.550829,2.086145,4.704612,1.021409,4.335701,0.973353,7.225902,6.248177,5.732063,1.432999,0.448819,4.071324,3.883944,2.190738,4.255717,5.636615,1.672150,5.847755,8.796700,6.136100,4.010981,2.648712,6.566596,3.705575,2.125699,7.886980,0.242161,7.096453,4.221281,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,84.386578,15.0,50.631

In [36]:
train_imputed.isna().sum().sum()

0

In [37]:
concat_df = pd.concat([train_imputed[test_imputed.columns], test_imputed], ignore_index=True)
print(concat_df.shape)
concat_df.head()

(2756, 171)


,Basic_Demos-Age,CGAS-CGAS_Score,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,Fitness_Endurance-Max_Stage,Fitness_Endurance-Time_Mins,Fitness_Endurance-Time_Sec,FGC-FGC_CU,FGC-FGC_CU_Zone,FGC-FGC_GSND,FGC-FGC_GSND_Zone,FGC-FGC_GSD,FGC-FGC_GSD_Zone,FGC-FGC_PU,FGC-FGC_PU_Zone,FGC-FGC_SRL,FGC-FGC_SRL_Zone,FGC-FGC_SRR,FGC-FGC_SRR_Zone,FGC-FGC_TL,FGC-FGC_TL_Zone,BIA-BIA_Activity_Level_num,BIA-BIA_BMC,BIA-BIA_BMI,BIA-BIA_BMR,BIA-BIA_DEE,BIA-BIA_ECW,BIA-BIA_FFM,BIA-BIA_FFMI,BIA-BIA_FMI,BIA-BIA_Fat,BIA-BIA_Frame_num,BIA-BIA_ICW,BIA-BIA_LDM,BIA-BIA_LST,BIA-BIA_SMM,BIA-BIA_TBW,PAQ_A-PAQ_A_Total,PAQ_C-PAQ_C_Total,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-computerinternet_hoursday,Enc_1,Enc_2,Enc_3,Enc_4,Enc_5,Enc_6,Enc_7,Enc_8,Enc_9,Enc_10,Enc_11,Enc_13,Enc_14,Enc_17,Enc_18,Enc_19,Enc_20,Enc_22,Enc_23,Enc_24,Enc_25,Enc_26,Enc_27,Enc_28,Enc_29,Enc_30,Enc_31,Enc_32,Enc_33,Enc_34,Enc_35,Enc_36,Enc_37,Enc_38,Enc_39,Enc_40,Enc_41,Enc_42,Enc_43,Enc_44,Enc_45,Enc_46,Enc_47,Enc_48,Enc_50,Enc_51,Enc_52,Enc_53,Enc_54,Enc_55,Enc_56,Enc_57,Enc_58,Enc_60,Enc_61,Enc_62,Enc_63,Enc_64,PAQ_A-Season_Fall,PAQ_A-Season_Missing,PAQ_A-Season_Spring,PAQ_A-Season_Summer,PAQ_A-Season_Winter,Basic_Demos-Enroll_Season_Fall,Basic_Demos-Enroll_Season_Spring,Basic_Demos-Enroll_Season_Summer,Basic_Demos-Enroll_Season_Winter,Fitness_Endurance-Season_Fall,Fitness_Endurance-Season_Missing,Fitness_Endurance-Season_Spring,Fitness_Endurance-Season_Summer,Fitness_Endurance-Season_Winter,SDS-Season_Fall,SDS-Season_Missing,SDS-Season_Spring,SDS-Season_Summer,SDS-Season_Winter,Basic_Demos-Sex_0,Basic_Demos-Sex_1,BIA-Season_Fall,BIA-Season_Missing,BIA-Season_Spring,BIA-Season_Summer,BIA-Season_Winter,PreInt_EduHx-Season_Fall,PreInt_EduHx-Season_Missing,PreInt_EduHx-Season_Spring,PreInt_EduHx-Season_Summer,PreInt_EduHx-Season_Winter,PAQ_C-Season_Fall,PAQ_C-Season_Missing,PAQ_C-Season_Spring,PAQ_C-Season_Summer,PAQ_C-Season_Winter,Physical-Season_Fall,Physical-Season_Missing,Physical-Season_Spring,Physical-Season_Summer,Physical-Season_Winter,CGAS-Season_Fall,CGAS-Season_Missing,CGAS-Season_Spring,CGAS-Season_Summer,CGAS-Season_Winter,FGC-Season_Fall,FGC-Season_Missing,FGC-Season_Spring,FGC-Season_Summer,FGC-Season_Winter,BMI_Age,Internet_Hours_Age,BMI_Internet_Hours,BFP_BMI,FFMI_BFP,FMI_BFP,LST_TBW,BFP_BMR,BFP_DEE,BMR_Weight,DEE_Weight,SMM_Height,Muscle_to_Fat,Hydration_Status,ICW_TBW
0,5.0,51.0,16.877316,46.0,50.8,24.2,66.8,79.4,109.6,4.6,7.4,16.8,0.0,0.0,14.44,1.2,15.06,1.4,0.0,0.0,7.0,0.0,6.0,0.0,6.0,1.0,2.0,2.668550,16.8792,932.498,1492.00,8.25598,41.58620,13.81770,3.061430,9.21377,1.0,24.43490,8.89536,38.91770,19.5413,32.69090,1.912,2.170,42.4,59.4,3.0,1.940226,7.090114,6.851843,3.312979,3.731173,4.773416,6.047329,0.303090,1.557901,0.172479,2.217920,4.374257,2.017492,7.408865,3.042647,3.437833,0.000000,8.375595,4.072908,2.629780,5.833642,2.703716,2.645724,8.372981,7.545446,7.182353,1.780870,5.257917,3.777040,5.550829,2.086145,4.704612,1.021409,4.335701,0.973353,7.225902,6.248177,5.732063,1.432999,0.448819,4.071324,3.883944,2.190738,4.255717,5.636615,1.672150,5.847755,8.796700,6.136100,4.010981,2.648712,6.566596,3.705575,2.125699,7.886980,0.242161,7.096453,4.221281,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,84.386578,15.0,50.631947,0.545865,1.499679,0.332267,1.190475,8591.822097,13746.944840,18.356260,29.370079,0.424811,6.383063,0.643522,0.747453
1,9.0,63.4,14.035590,48.0,46.0,22.0,75.0,70.0,122.0,5.4,8.4,19.2,3.0,0.0,21.82,1.8,24.44,2.0,5.0,0.0,11.0,1.0,11.0,1.0,3.0,0.0,2.0,2.579490,14.0371,936.656,1498.65,6.01993,42.02910,12.82540,1.211720,3.97085,1.0,21.03520,14.97400,39.44970,15.4107,27.05520,1.582,2.340,46.0,64.0,0.0,2.478349,3.097783,4.403833,4.813544,3.082445,6.671233,4.776706,0.914687,0.789561,3.136578,2.443026,0.59

In [38]:
imputer = KNNImputer(n_neighbors=5)
imputed_data_concat = imputer.fit_transform(concat_df)

In [39]:
concat_imputed = pd.DataFrame(imputed_data_concat, columns=concat_df.columns)
print(concat_imputed.shape)
concat_imputed.head()

(2756, 171)


,Basic_Demos-Age,CGAS-CGAS_Score,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,Fitness_Endurance-Max_Stage,Fitness_Endurance-Time_Mins,Fitness_Endurance-Time_Sec,FGC-FGC_CU,FGC-FGC_CU_Zone,FGC-FGC_GSND,FGC-FGC_GSND_Zone,FGC-FGC_GSD,FGC-FGC_GSD_Zone,FGC-FGC_PU,FGC-FGC_PU_Zone,FGC-FGC_SRL,FGC-FGC_SRL_Zone,FGC-FGC_SRR,FGC-FGC_SRR_Zone,FGC-FGC_TL,FGC-FGC_TL_Zone,BIA-BIA_Activity_Level_num,BIA-BIA_BMC,BIA-BIA_BMI,BIA-BIA_BMR,BIA-BIA_DEE,BIA-BIA_ECW,BIA-BIA_FFM,BIA-BIA_FFMI,BIA-BIA_FMI,BIA-BIA_Fat,BIA-BIA_Frame_num,BIA-BIA_ICW,BIA-BIA_LDM,BIA-BIA_LST,BIA-BIA_SMM,BIA-BIA_TBW,PAQ_A-PAQ_A_Total,PAQ_C-PAQ_C_Total,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-computerinternet_hoursday,Enc_1,Enc_2,Enc_3,Enc_4,Enc_5,Enc_6,Enc_7,Enc_8,Enc_9,Enc_10,Enc_11,Enc_13,Enc_14,Enc_17,Enc_18,Enc_19,Enc_20,Enc_22,Enc_23,Enc_24,Enc_25,Enc_26,Enc_27,Enc_28,Enc_29,Enc_30,Enc_31,Enc_32,Enc_33,Enc_34,Enc_35,Enc_36,Enc_37,Enc_38,Enc_39,Enc_40,Enc_41,Enc_42,Enc_43,Enc_44,Enc_45,Enc_46,Enc_47,Enc_48,Enc_50,Enc_51,Enc_52,Enc_53,Enc_54,Enc_55,Enc_56,Enc_57,Enc_58,Enc_60,Enc_61,Enc_62,Enc_63,Enc_64,PAQ_A-Season_Fall,PAQ_A-Season_Missing,PAQ_A-Season_Spring,PAQ_A-Season_Summer,PAQ_A-Season_Winter,Basic_Demos-Enroll_Season_Fall,Basic_Demos-Enroll_Season_Spring,Basic_Demos-Enroll_Season_Summer,Basic_Demos-Enroll_Season_Winter,Fitness_Endurance-Season_Fall,Fitness_Endurance-Season_Missing,Fitness_Endurance-Season_Spring,Fitness_Endurance-Season_Summer,Fitness_Endurance-Season_Winter,SDS-Season_Fall,SDS-Season_Missing,SDS-Season_Spring,SDS-Season_Summer,SDS-Season_Winter,Basic_Demos-Sex_0,Basic_Demos-Sex_1,BIA-Season_Fall,BIA-Season_Missing,BIA-Season_Spring,BIA-Season_Summer,BIA-Season_Winter,PreInt_EduHx-Season_Fall,PreInt_EduHx-Season_Missing,PreInt_EduHx-Season_Spring,PreInt_EduHx-Season_Summer,PreInt_EduHx-Season_Winter,PAQ_C-Season_Fall,PAQ_C-Season_Missing,PAQ_C-Season_Spring,PAQ_C-Season_Summer,PAQ_C-Season_Winter,Physical-Season_Fall,Physical-Season_Missing,Physical-Season_Spring,Physical-Season_Summer,Physical-Season_Winter,CGAS-Season_Fall,CGAS-Season_Missing,CGAS-Season_Spring,CGAS-Season_Summer,CGAS-Season_Winter,FGC-Season_Fall,FGC-Season_Missing,FGC-Season_Spring,FGC-Season_Summer,FGC-Season_Winter,BMI_Age,Internet_Hours_Age,BMI_Internet_Hours,BFP_BMI,FFMI_BFP,FMI_BFP,LST_TBW,BFP_BMR,BFP_DEE,BMR_Weight,DEE_Weight,SMM_Height,Muscle_to_Fat,Hydration_Status,ICW_TBW
0,5.0,51.0,16.877316,46.0,50.8,24.2,66.8,79.4,109.6,4.6,7.4,16.8,0.0,0.0,14.44,1.2,15.06,1.4,0.0,0.0,7.0,0.0,6.0,0.0,6.0,1.0,2.0,2.668550,16.8792,932.498,1492.00,8.25598,41.58620,13.81770,3.061430,9.21377,1.0,24.43490,8.89536,38.91770,19.5413,32.69090,1.912,2.170,42.4,59.4,3.0,1.940226,7.090114,6.851843,3.312979,3.731173,4.773416,6.047329,0.303090,1.557901,0.172479,2.217920,4.374257,2.017492,7.408865,3.042647,3.437833,0.000000,8.375595,4.072908,2.629780,5.833642,2.703716,2.645724,8.372981,7.545446,7.182353,1.780870,5.257917,3.777040,5.550829,2.086145,4.704612,1.021409,4.335701,0.973353,7.225902,6.248177,5.732063,1.432999,0.448819,4.071324,3.883944,2.190738,4.255717,5.636615,1.672150,5.847755,8.796700,6.136100,4.010981,2.648712,6.566596,3.705575,2.125699,7.886980,0.242161,7.096453,4.221281,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,84.386578,15.0,50.631947,0.545865,1.499679,0.332267,1.190475,8591.822097,13746.944840,18.356260,29.370079,0.424811,6.383063,0.643522,0.747453
1,9.0,63.4,14.035590,48.0,46.0,22.0,75.0,70.0,122.0,5.4,8.4,19.2,3.0,0.0,21.82,1.8,24.44,2.0,5.0,0.0,11.0,1.0,11.0,1.0,3.0,0.0,2.0,2.579490,14.0371,936.656,1498.65,6.01993,42.02910,12.82540,1.211720,3.97085,1.0,21.03520,14.97400,39.44970,15.4107,27.05520,1.582,2.340,46.0,64.0,0.0,2.478349,3.097783,4.403833,4.813544,3.082445,6.671233,4.776706,0.914687,0.789561,3.136578,2.443026,0.59

In [40]:
concat_imputed.isna().sum().sum()

0

In [41]:
test_imputed = concat_imputed.tail(test_imputed.shape[0])
print(test_imputed.shape)
test_imputed.head()

(20, 171)


,Basic_Demos-Age,CGAS-CGAS_Score,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,Fitness_Endurance-Max_Stage,Fitness_Endurance-Time_Mins,Fitness_Endurance-Time_Sec,FGC-FGC_CU,FGC-FGC_CU_Zone,FGC-FGC_GSND,FGC-FGC_GSND_Zone,FGC-FGC_GSD,FGC-FGC_GSD_Zone,FGC-FGC_PU,FGC-FGC_PU_Zone,FGC-FGC_SRL,FGC-FGC_SRL_Zone,FGC-FGC_SRR,FGC-FGC_SRR_Zone,FGC-FGC_TL,FGC-FGC_TL_Zone,BIA-BIA_Activity_Level_num,BIA-BIA_BMC,BIA-BIA_BMI,BIA-BIA_BMR,BIA-BIA_DEE,BIA-BIA_ECW,BIA-BIA_FFM,BIA-BIA_FFMI,BIA-BIA_FMI,BIA-BIA_Fat,BIA-BIA_Frame_num,BIA-BIA_ICW,BIA-BIA_LDM,BIA-BIA_LST,BIA-BIA_SMM,BIA-BIA_TBW,PAQ_A-PAQ_A_Total,PAQ_C-PAQ_C_Total,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-computerinternet_hoursday,Enc_1,Enc_2,Enc_3,Enc_4,Enc_5,Enc_6,Enc_7,Enc_8,Enc_9,Enc_10,Enc_11,Enc_13,Enc_14,Enc_17,Enc_18,Enc_19,Enc_20,Enc_22,Enc_23,Enc_24,Enc_25,Enc_26,Enc_27,Enc_28,Enc_29,Enc_30,Enc_31,Enc_32,Enc_33,Enc_34,Enc_35,Enc_36,Enc_37,Enc_38,Enc_39,Enc_40,Enc_41,Enc_42,Enc_43,Enc_44,Enc_45,Enc_46,Enc_47,Enc_48,Enc_50,Enc_51,Enc_52,Enc_53,Enc_54,Enc_55,Enc_56,Enc_57,Enc_58,Enc_60,Enc_61,Enc_62,Enc_63,Enc_64,PAQ_A-Season_Fall,PAQ_A-Season_Missing,PAQ_A-Season_Spring,PAQ_A-Season_Summer,PAQ_A-Season_Winter,Basic_Demos-Enroll_Season_Fall,Basic_Demos-Enroll_Season_Spring,Basic_Demos-Enroll_Season_Summer,Basic_Demos-Enroll_Season_Winter,Fitness_Endurance-Season_Fall,Fitness_Endurance-Season_Missing,Fitness_Endurance-Season_Spring,Fitness_Endurance-Season_Summer,Fitness_Endurance-Season_Winter,SDS-Season_Fall,SDS-Season_Missing,SDS-Season_Spring,SDS-Season_Summer,SDS-Season_Winter,Basic_Demos-Sex_0,Basic_Demos-Sex_1,BIA-Season_Fall,BIA-Season_Missing,BIA-Season_Spring,BIA-Season_Summer,BIA-Season_Winter,PreInt_EduHx-Season_Fall,PreInt_EduHx-Season_Missing,PreInt_EduHx-Season_Spring,PreInt_EduHx-Season_Summer,PreInt_EduHx-Season_Winter,PAQ_C-Season_Fall,PAQ_C-Season_Missing,PAQ_C-Season_Spring,PAQ_C-Season_Summer,PAQ_C-Season_Winter,Physical-Season_Fall,Physical-Season_Missing,Physical-Season_Spring,Physical-Season_Summer,Physical-Season_Winter,CGAS-Season_Fall,CGAS-Season_Missing,CGAS-Season_Spring,CGAS-Season_Summer,CGAS-Season_Winter,FGC-Season_Fall,FGC-Season_Missing,FGC-Season_Spring,FGC-Season_Summer,FGC-Season_Winter,BMI_Age,Internet_Hours_Age,BMI_Internet_Hours,BFP_BMI,FFMI_BFP,FMI_BFP,LST_TBW,BFP_BMR,BFP_DEE,BMR_Weight,DEE_Weight,SMM_Height,Muscle_to_Fat,Hydration_Status,ICW_TBW
2736,5.0,51.00,16.877316,46.0,50.800,23.64,71.24,78.60,121.52,4.44,6.56,26.52,0.00,0.00,15.900,1.60,16.744,1.68,0.00,0.00,7.000,0.00,6.00,0.00,6.00,1.0,2.00,2.668550,16.879200,932.4980,1492.0000,8.255980,41.586200,13.817700,3.061430,9.213770,1.00,24.434900,8.895360,38.917700,19.541300,32.690900,1.75440,2.5612,39.88,56.28,3.0,4.146095,5.098936,5.751642,5.369324,1.907452,7.346219,3.697630,0.072829,2.082264,3.762738,1.974344,3.015886,4.338364,4.126766,3.388199,6.420395,5.401684,10.703547,3.947854,5.171425,4.675375,2.711288,4.898180,4.027126,7.029704,10.801527,3.229619,11.602550,3.051257,8.731516,3.972671,3.518056,3.558551,2.442623,3.994294,4.141173,4.263391,5.596790,4.488864,0.523367,2.594709,2.421323,1.695968,6.214463,8.031522,5.475738,3.961531,7.407165,10.539744,2.188900,3.814482,9.613784,4.796636,3.670116,4.546722,4.561527,2.203451,4.299928,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,84.386578,15.0,50.631947,0.545865,1.499679,0.332267,1.190475,8591.822097,13746.944840,18.356260,29.370079,0.424811,6.383063,0.643522,0.747453
2737,9.0,58.48,14.035590,48.0,46.000,22.00,75.00,70.00,122.00,5.04,7.60,27.40,3.00,0.00,16.764,1.64,17.708,1.72,5.00,0.00,11.000,1.00,11.00,1.00,3.00,0.0,2.00,2.579490,14.037100,936.6560,1498.6500,6.019930,42.029100,12.825400,1.211720,3.970850,1.00,21.035200,14.974000,39.449700,15.410700,27.055200,1.54720,2.3400,46.00,64.00,0.0,6

In [42]:
test_imputed.isna().sum().sum()

0

In [43]:
train_final = train_imputed[test_imputed.columns.tolist() + ['sii']].copy()
test_final = test_imputed.copy()

In [44]:
print(train_final.shape)
print(test_final.shape)

(2736, 172)
(20, 171)


In [45]:
def quadratic_weighted_kappa(y_true, y_pred):
    return cohen_kappa_score(y_true, y_pred, weights='quadratic')

def threshold_Rounder(oof_non_rounded, thresholds):
    return np.where(oof_non_rounded < thresholds[0], 0,
                    np.where(oof_non_rounded < thresholds[1], 1,
                             np.where(oof_non_rounded < thresholds[2], 2, 3)))

def evaluate_predictions(thresholds, y_true, oof_non_rounded):
    rounded_p = threshold_Rounder(oof_non_rounded, thresholds)
    return -quadratic_weighted_kappa(y_true, rounded_p)

# Model Training and Evaluation

- **Model Types**: Various models are used, including:
  - **LightGBM**: A gradient-boosting framework known for its speed and efficiency with large datasets.
  - **XGBoost**: Another powerful gradient-boosting model used for structured data.
  - **CatBoost**: Optimized for categorical features without the need for extensive preprocessing.
  - **Voting Regressor**: An ensemble model that combines the predictions of LightGBM, XGBoost, and CatBoost for better accuracy.
- **Cross-Validation**: Stratified K-Folds cross-validation is employed to split the data into training and validation sets, ensuring balanced class distribution in each fold.
- **Quadratic Weighted Kappa (QWK)**: The performance of the models is evaluated using QWK, which measures the agreement between predicted and actual values, taking into account the ordinal nature of the target variable.
- **Threshold Optimization**: The `minimize` function from `scipy.optimize` is used to fine-tune decision thresholds that map continuous predictions to discrete categories (None, Mild, Moderate, Severe).


In [46]:
def TrainML(model_class, train_data, test_data):
    X = train_data.drop(['sii'], axis=1)
    y = train_data['sii']

    SKF = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=SEED)
    
    train_S = []
    test_S = []
    
    oof_non_rounded = np.zeros(len(y), dtype=float) 
    oof_rounded = np.zeros(len(y), dtype=int) 
    test_preds = np.zeros((len(test_data), n_splits))

    for fold, (train_idx, test_idx) in enumerate(tqdm(SKF.split(X, y), desc="Training Folds", total=n_splits)):
        X_train, X_val = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[test_idx]

        model = clone(model_class)
        model.fit(X_train, y_train)

        y_train_pred = model.predict(X_train)
        y_val_pred = model.predict(X_val)

        oof_non_rounded[test_idx] = y_val_pred
        y_val_pred_rounded = y_val_pred.round(0).astype(int)
        oof_rounded[test_idx] = y_val_pred_rounded

        train_kappa = quadratic_weighted_kappa(y_train, y_train_pred.round(0).astype(int))
        val_kappa = quadratic_weighted_kappa(y_val, y_val_pred_rounded)

        train_S.append(train_kappa)
        test_S.append(val_kappa)
        
        test_preds[:, fold] = model.predict(test_data)
        
        print(f"Fold {fold+1} - Train QWK: {train_kappa:.4f}, Validation QWK: {val_kappa:.4f}")
        clear_output(wait=True)

    print(f"Mean Train QWK --> {np.mean(train_S):.4f}")
    print(f"Mean Validation QWK ---> {np.mean(test_S):.4f}")

    KappaOPtimizer = minimize(evaluate_predictions,
                              x0=[0.5, 1.5, 2.5], args=(y, oof_non_rounded), 
                              method='Nelder-Mead')
    assert KappaOPtimizer.success, "Optimization did not converge."
    
    oof_tuned = threshold_Rounder(oof_non_rounded, KappaOPtimizer.x)
    tKappa = quadratic_weighted_kappa(y, oof_tuned)

    print(f"----> || Optimized QWK SCORE :: {Fore.CYAN}{Style.BRIGHT} {tKappa:.3f}{Style.RESET_ALL}")

    tpm = test_preds.mean(axis=1)
    tpTuned = threshold_Rounder(tpm, KappaOPtimizer.x)
    
    submission = pd.DataFrame({
        'id': sample['id'],
        'sii': tpTuned
    })

    return submission


# Hyperparameter Tuning

- **LightGBM Parameters**: Hyperparameters such as `learning_rate`, `max_depth`, `num_leaves`, and `feature_fraction` are tuned to improve the performance of the LightGBM model. These parameters control the complexity of the model and its ability to generalize to new data.
- **XGBoost and CatBoost Parameters**: Similar tuning is applied for XGBoost and CatBoost, adjusting parameters such as `n_estimators`, `max_depth`, `learning_rate`, `subsample`, and `regularization` terms (`reg_alpha`, `reg_lambda`). These help in controlling overfitting and ensuring the model's robustness.

In [47]:
# Model parameters for LightGBM
Params = {
    'learning_rate': 0.046,
    'max_depth': 12,
    'num_leaves': 478,
    'min_data_in_leaf': 13,
    'feature_fraction': 0.893,
    'bagging_fraction': 0.784,
    'bagging_freq': 4,
    'lambda_l1': 10,  # Increased from 6.59
    'lambda_l2': 0.01,  # Increased from 2.68e-06
    'device': 'gpu'

}


# XGBoost parameters
XGB_Params = {
    'learning_rate': 0.05,
    'max_depth': 6,
    'n_estimators': 200,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'reg_alpha': 1,  # Increased from 0.1
    'reg_lambda': 5,  # Increased from 1
    'random_state': SEED,
    'tree_method': 'gpu_hist',

}


CatBoost_Params = {
    'learning_rate': 0.05,
    'depth': 6,
    'iterations': 200,
    'random_seed': SEED,
    'verbose': 0,
    'l2_leaf_reg': 10,  # Increase this value
    'task_type': 'GPU'

}

In [48]:
# New: TabNet

from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.model_selection import train_test_split
from pytorch_tabnet.callbacks import Callback

class TabNetWrapper(BaseEstimator, RegressorMixin):
    def __init__(self, **kwargs):
        self.model = TabNetRegressor(**kwargs)
        self.kwargs = kwargs
        self.imputer = SimpleImputer(strategy='median')
        self.best_model_path = 'best_tabnet_model.pt'
        
    def fit(self, X, y):
        # Handle missing values
        X_imputed = self.imputer.fit_transform(X)
        
        if hasattr(y, 'values'):
            y = y.values
            
        # Create internal validation set
        X_train, X_valid, y_train, y_valid = train_test_split(
            X_imputed, 
            y, 
            test_size=0.2,
            random_state=42
        )
        
        # Train TabNet model
        history = self.model.fit(
            X_train=X_train,
            y_train=y_train.reshape(-1, 1),
            eval_set=[(X_valid, y_valid.reshape(-1, 1))],
            eval_name=['valid'],
            eval_metric=['mse'],
            max_epochs=500,
            patience=50,
            batch_size=1024,
            virtual_batch_size=128,
            num_workers=0,
            drop_last=False,
            callbacks=[
                TabNetPretrainedModelCheckpoint(
                    filepath=self.best_model_path,
                    monitor='valid_mse',
                    mode='min',
                    save_best_only=True,
                    verbose=True
                )
            ]
        )
        
        # Load the best model
        if os.path.exists(self.best_model_path):
            self.model.load_model(self.best_model_path)
            os.remove(self.best_model_path)  # Remove temporary file
        
        return self
    
    def predict(self, X):
        X_imputed = self.imputer.transform(X)
        return self.model.predict(X_imputed).flatten()
    
    def __deepcopy__(self, memo):
        # Add deepcopy support for scikit-learn
        cls = self.__class__
        result = cls.__new__(cls)
        memo[id(self)] = result
        for k, v in self.__dict__.items():
            setattr(result, k, deepcopy(v, memo))
        return result

# TabNet hyperparameters
TabNet_Params = {
    'n_d': 64,              # Width of the decision prediction layer
    'n_a': 64,              # Width of the attention embedding for each step
    'n_steps': 5,           # Number of steps in the architecture
    'gamma': 1.5,           # Coefficient for feature selection regularization
    'n_independent': 2,     # Number of independent GLU layer in each GLU block
    'n_shared': 2,          # Number of shared GLU layer in each GLU block
    'lambda_sparse': 1e-4,  # Sparsity regularization
    'optimizer_fn': torch.optim.Adam,
    'optimizer_params': dict(lr=2e-2, weight_decay=1e-5),
    'mask_type': 'entmax',
    'scheduler_params': dict(mode="min", patience=10, min_lr=1e-5, factor=0.5),
    'scheduler_fn': torch.optim.lr_scheduler.ReduceLROnPlateau,
    'verbose': 1,
    'device_name': 'cuda' if torch.cuda.is_available() else 'cpu'
}

class TabNetPretrainedModelCheckpoint(Callback):
    def __init__(self, filepath, monitor='val_loss', mode='min', 
                 save_best_only=True, verbose=1):
        super().__init__()  # Initialize parent class
        self.filepath = filepath
        self.monitor = monitor
        self.mode = mode
        self.save_best_only = save_best_only
        self.verbose = verbose
        self.best = float('inf') if mode == 'min' else -float('inf')
        
    def on_train_begin(self, logs=None):
        self.model = self.trainer  # Use trainer itself as model
        
    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        current = logs.get(self.monitor)
        if current is None:
            return
        
        # Check if current metric is better than best
        if (self.mode == 'min' and current < self.best) or \
           (self.mode == 'max' and current > self.best):
            if self.verbose:
                print(f'\nEpoch {epoch}: {self.monitor} improved from {self.best:.4f} to {current:.4f}')
            self.best = current
            if self.save_best_only:
                self.model.save_model(self.filepath)  # Save the entire model

# Ensemble Learning and Submission Preparation

- **Ensemble Learning**: The model uses a **Voting Regressor**, which combines the predictions from LightGBM, XGBoost, and CatBoost. This approach is beneficial as it leverages the strengths of multiple models, reducing overfitting and improving overall model performance.
- **Out-of-Fold (OOF) Predictions**: During cross-validation, out-of-fold predictions are generated for the training set, which helps in model evaluation without data leakage.
- **Kappa Optimizer**: The Kappa Optimizer ensures that the predicted values are as close to the actual values as possible by adjusting the thresholds used to convert raw model outputs into class labels.
- **Test Set Predictions**: After the model is trained and thresholds are optimized, the test dataset is processed, and predictions are generated using the ensemble model. These predictions are converted into the appropriate format for submission.
- **Submission File Creation**: The predictions are saved in a CSV file following the required format for submission (e.g., for a Kaggle competition), which includes columns like `id` and `sii` (Severity Impairment Index).

# Final Results and Performance Metrics

- **Train and Validation Scores**: After training across multiple folds, the mean Quadratic Weighted Kappa (QWK) score is calculated for both the training and validation datasets, providing an indicator of model performance. 
- **Optimized QWK Score**: The final optimized QWK score after threshold tuning is displayed, showcasing the model's ability to predict the severity levels effectively.
- **Test Predictions**: The test set predictions are evaluated, and a breakdown of the predicted severity levels (None, Mild, Moderate, Severe) is shown, along with their respective counts.

In [49]:
# Create model instances
Light = LGBMRegressor(**Params, random_state=SEED, verbose=-1, n_estimators=300)
XGB_Model = XGBRegressor(**XGB_Params)
CatBoost_Model = CatBoostRegressor(**CatBoost_Params)
TabNet_Model = TabNetWrapper(**TabNet_Params) # New

voting_model = VotingRegressor(estimators=[
    ('lightgbm', Light),
    ('xgboost', XGB_Model),
    ('catboost', CatBoost_Model),
    ('tabnet', TabNet_Model)
])

In [50]:
Submission1 = TrainML(voting_model, train_final, test_final)

Submission1

Training Folds: 100%|████████████████████████████████████████████████████████████████████| 5/5 [02:28<00:00, 29.61s/it]

Mean Train QWK --> 0.8858
Mean Validation QWK ---> 0.6315
----> || Optimized QWK SCORE ::  0.678


,id,sii
0,00008ff9,0
1,000fd460,0
2,00105258,2
3,00115b9f,1
4,0016bb22,1
5,001f3379,1
6,0038ba98,1
7,0068a485,2
8,0069fbed,2
9,0083e397,1


In [51]:
imputer = SimpleImputer(strategy='median')

ensemble = VotingRegressor(estimators=[
    ('lgb', Pipeline(steps=[('imputer', imputer), ('regressor', LGBMRegressor(random_state=SEED))])),
    ('xgb', Pipeline(steps=[('imputer', imputer), ('regressor', XGBRegressor(random_state=SEED))])),
    ('cat', Pipeline(steps=[('imputer', imputer), ('regressor', CatBoostRegressor(random_state=SEED, silent=True))])),
    ('rf', Pipeline(steps=[('imputer', imputer), ('regressor', RandomForestRegressor(random_state=SEED))])),
    ('gb', Pipeline(steps=[('imputer', imputer), ('regressor', GradientBoostingRegressor(random_state=SEED))])),
    ('tabnet', Pipeline(steps=[('imputer', imputer), ('regressor', TabNetWrapper(**TabNet_Params))]))  # New:TabNet
])

In [52]:
Submission3 = TrainML(ensemble, train_final, test_final)

Submission3

Training Folds: 100%|████████████████████████████████████████████████████████████████████| 5/5 [03:50<00:00, 46.01s/it]

Mean Train QWK --> 0.9764
Mean Validation QWK ---> 0.6434
----> || Optimized QWK SCORE ::  0.681


,id,sii
0,00008ff9,0
1,000fd460,0
2,00105258,2
3,00115b9f,1
4,0016bb22,0
5,001f3379,1
6,0038ba98,1
7,0068a485,2
8,0069fbed,2
9,0083e397,1


In [53]:
sub1 = Submission1
sub2 = Submission2
sub3 = Submission3

sub1 = sub1.sort_values(by='id').reset_index(drop=True)
sub2 = sub2.sort_values(by='id').reset_index(drop=True)
sub3 = sub3.sort_values(by='id').reset_index(drop=True)

combined = pd.DataFrame({
    'id': sub1['id'],
    'sii_1': sub1['sii'],
    'sii_2': sub2['sii'],
    'sii_3': sub3['sii']
})

def majority_vote(row):
    return row.mode()[0]

combined['final_sii'] = combined[['sii_1', 'sii_2', 'sii_3']].apply(majority_vote, axis=1)

final_submission = combined[['id', 'final_sii']].rename(columns={'final_sii': 'sii'})

final_submission.to_csv('submission.csv', index=False)

print("Majority voting completed and saved to 'Final_Submission.csv'")

NameError: name 'Submission2' is not defined

In [ ]:
final_submission